In [5]:
%reset -f

In [1]:
from EEG2CodeKeras import (basearchi,
                           basearchitest_batchnorm,
                           basearchi_patchembedding,
                           basearchi_patchembeddingdilation,
                           trueVanilliaEEG2Code,
                           vanilliaEEG2Code,
                           vanilliaEEG2Code2,
                           EEGnet_Inception)
from _utils import make_preds_accumul_aggresive, make_preds_pvalue


from sklearn.model_selection import train_test_split
from sklearn.base import clone
from sklearn.pipeline import make_pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import balanced_accuracy_score,confusion_matrix
from sklearn.cross_decomposition import CCA


from pyriemann.estimation import XdawnCovariances, Xdawn
from pyriemann.estimation import Covariances
from pyriemann.utils.distance import distance_riemann 
from pyriemann.tangentspace import TangentSpace
from mne.decoding import Vectorizer
from pyriemann.transfer import (
    decode_domains,
    encode_domains,
    TLCenter,
    TLRotate,
)

from tensorflow import keras
from collections import OrderedDict
import tensorflow as tf
import mne
import os

import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import sys
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score
np.set_printoptions(threshold=sys.maxsize)
import matplotlib.pyplot as plt



c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.15.0 and is not supported. 
Some things might work, some things might not.
If you were to en

In [2]:
from moabb.paradigms import CVEP
from Scripts.SPDNet.tensorflow.spd_net_2_tensorflow import SPDNet_AJD
from Scripts.SPDNet.tensorflow.spd_net_tensorflow import SPDNet_Tensorflow
from Scripts.SPDNet.tensorflow.optimizer import riemannian_adam
from Scripts.SPDNet.torch.optimizers import riemannian_adam as torch_riemannian_adam
from Scripts.SPDNet.torch.spd_net_bn_torch import SPDNetBN_Torch, SPDNetBN_Module
from sklearn.model_selection import GridSearchCV
sys.path.insert(0,"C:\\Users\\s.velut\\Documents\\These\\moabb\\moabb\\datasets")
sys.path.insert(0,"C:\\Users\\s.velut\\Documents\\These\\moabb\\moabb\\paradigms")
sys.path.insert(0,"C:\\Users\\s.velut\\Documents\\These\\Protheus_PHD\\Scripts\\SPDNet")
sys.path.insert(0,"C:\\Users\\s.velut\\Documents\\These\\riemannian_tSNE")
from R_TSNE import R_TSNE
from Scripts.utils import prepare_data,get_BVEP_data,balance,get_y_pred
from castillos2023 import CasitllosCVEP100,CasitllosCVEP40,CasitllosBurstVEP100,CasitllosBurstVEP40

# get the functions from RPA package
sys.path.insert(0,"C:\\Users\\s.velut\\Documents\\These\\RPA")
import rpa.transfer_learning as TL
import rpa.diffusion_map as DM
import rpa.get_dataset as GD

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\moabb\pipelines\__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(


Instructions for updating:
experimental_compile is deprecated, use jit_compile instead


In [3]:
fps = 60
sfreq = 500


## LOAD THE DATA

In [45]:
dataset_moabb = CasitllosBurstVEP100()
paradigm = CVEP()
print(paradigm.n_classes)

raw = dataset_moabb.get_data()
print(raw)

Choosing the first None classes from all possible events.


None
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped


C:\Users\s.velut\Documents\These\moabb\moabb\datasets\castillos2023.py:182: RuntimeWarning: Data file name in EEG.data (P13_burst100.fdt) is incorrect, the file name must have changed on disk, using the correct file name (P6_burst100.fdt).
  raw = mne.io.read_raw_eeglab(file_path_list[0],preload=True, verbose=False)


Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
{1: {'0': {'0': <RawEEGLAB | P1_burst100.fdt, 34 x 185508 (371.0 s), ~48.2 MB, data loaded>}}, 2: {'0': {'0': <RawEEGLAB | P2_burst100.fdt, 34 x 124531 (249.1 s), ~32.3 MB, data loaded>}}, 3: {'0': {'0': <RawEEGLAB | P3_burst100.fdt, 34 x 131337 (262.7 s), ~34.1 MB, data loaded>}}, 4: {'0': {'0': <RawEEGLAB | P4_burst100.fdt, 34 x 134765 (269.5 s), ~35.0 MB, data loaded>}}, 5: {'0': {'0': <RawEEGLAB | P

In [46]:
raw[1]["0"]["0"]

<RawEEGLAB | P1_burst100.fdt, 34 x 185508 (371.0 s), ~48.2 MB, data loaded>

In [4]:
%matplotlib Qt

In [5]:
window_size=0.25
n_class=4
n_samples_windows = int(window_size*sfreq)

In [5]:
n_trial_per_class = int(len(raw)/n_class)

NameError: name 'raw' is not defined

## Get raw data

In [50]:
raw_data = []
keys = list(raw.keys())
labels = []
labels_code = []
to_keep = False

for i in range(12):
    temp = raw[keys[i]]["0"]["0"]

    temp_raw = temp.copy()
    trial_chan = temp_raw.pick_channels(["stim_trial"],verbose=False)
    data = trial_chan.get_data()[0]
    labels_code.append(np.array(list(filter(lambda num: num != 0, data)),dtype=int)-200)
    if to_keep:
        keep = ["O1", "O2", "Oz", "P7", "P3", "P4", "P8", "Pz","stim_epoch","stim_trial"]
        temp = temp.drop_channels([i for i in temp.ch_names if i not in keep])
    temp = temp.filter(l_freq=1, h_freq=25, method="fir", verbose=True)
    mne.set_eeg_reference(temp, 'average', copy=False, verbose=False)
    events = mne.find_events(temp,["stim_epoch"])
    epochs = mne.Epochs(temp,events,{"0":100,"1":101},picks=temp.ch_names[:-2],tmin=-0.01,tmax=window_size)
    labels.append(epochs.events[...,-1]-100)
    raw_data.append(epochs.get_data())

raw_data = np.array(raw_data)
labels = np.array(labels)
labels_code = np.array(labels_code)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 1651 samples (3.302 s)



7020 events found
Event IDs: [100 101]
Not setting metadata
7020 matching events found
Setting baseline interval to [-0.01, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 7020 events and 131 original time points ...


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


0 bad epochs dropped
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 1651 samples (3.302 s)

7020 events found
Event IDs: [100 101]
Not setting metadata
7020 matching events found
Setting baseline interval to [-0.01, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 7020 events and 131 original time points ...


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


...

0 bad epochs dropped
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 1651 samples (3.302 s)

7020 events found
Event IDs: [100 101]
Not setting metadata
7020 matching events found
Setting baseline interval to [-0.01, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 7020 events and 131 original time points ...


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


0 bad epochs dropped
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 1651 samples (3.302 s)

7020 events found
Event IDs: [100 101]
Not setting metadata
7020 matching events found
Setting baseline interval to [-0.01, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 7020 events and 131 original time points ...


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


0 bad epochs dropped


In [51]:
raw_data.shape

(12, 7020, 32, 131)

## Classifier : CCA

### LOO

In [ ]:
accuracy_code_looa = np.zeros(12)
tps_code_looa = np.zeros(12)
accuracy_looa = np.zeros(12)

for i in range(12):
    print("TL to the participant : ", i)
    ind2take = [j for j in range(12) if j!=i]
    X_train = raw_data[ind2take].reshape(-1,raw_data.shape[-2]*raw_data.shape[-1])
    Y_train = labels[ind2take].reshape(-1)
    Y_train = np.vstack((Y_train,np.abs(1-Y_train))).T
    X_test = raw_data[i].reshape(-1,raw_data.shape[-2]*raw_data.shape[-1])
    Y_test = labels[i]
    Y_test = np.vstack((Y_test,np.abs(1-Y_test))).T
    labels_code_test = labels_code[i]

    print(X_train.shape)
    print(X_test.shape)
    X_std = X_train.std(axis=0)
    X_train /= X_std + 1e-8
    X_std = X_test.std(axis=0)
    X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    rus = RandomUnderSampler()
    counter=np.array(range(0,len(Y_train))).reshape(-1,1)
    index,_ = rus.fit_resample(counter,Y_train[:])
    X_train = np.squeeze(X_train[index,:], axis=1)
    Y_train = np.squeeze(Y_train[index])

    print("Creating the different pipelines")
    clf = make_pipeline(CCA(n_components=2))
    print("Fitting")
    clf = clf.fit(np.array(X_train), Y_train)
    y_pred = clf.predict(X_test)
    y_pred = np.array([1 if (y >= 0.5) else 0 for y in y_pred[:,0]])
    y_test_norm = np.array([0 if y[0] == 0 else 1 for y in Y_test])
    print("getting accuracy of participant ", i)
    accuracy_looa[i] = balanced_accuracy_score(y_test_norm,y_pred)

    labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
            y_pred, dataset_moabb.codes, min_len=30, sfreq=60, consecutive=50, window_size=window_size
    )
    tps_code_looa[i] = np.mean(mean_long_accumul)
    accuracy_code_looa[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(accuracy_looa)
print(accuracy_code_looa)
print(tps_code_looa)


### Adaptative LOO

In [ ]:
n_cal = 4
accuracy_code_adap = np.zeros(12)
tps_code_adap = np.zeros(12)
accuracy_adap = np.zeros(12)

for i in range(12):
    print("TL to the participant : ", i)
    ind2take = [j for j in range(12) if j!=i]
    X_source = raw_data[ind2take].reshape(-1,raw_data.shape[-2]*raw_data.shape[-1])
    X_target = raw_data[i].reshape(-1,raw_data.shape[-2]*raw_data.shape[-1])
    X_train = np.concatenate([X_source,X_target[:int((2.2-window_size)*n_class*n_cal*60)]],axis=0)
    Y_train = np.concatenate([labels[ind2take].reshape(-1),labels[i][:int((2.2-window_size)*n_class*n_cal*60)]],axis=0)
    Y_train = np.vstack((Y_train,np.abs(1-Y_train))).T
    
    Y_test = labels[i][int((2.2-window_size)*n_class*n_cal*60):]
    X_test = X_target[int((2.2-window_size)*n_class*n_cal*60):]
    Y_test = np.vstack((Y_test,np.abs(1-Y_test))).T

    labels_code_test = labels_code[i][n_cal*4:]

    print(X_train.shape)
    print(X_test.shape)
    X_std = X_train.std(axis=0)
    X_train /= X_std + 1e-8
    X_std = X_test.std(axis=0)
    X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    rus = RandomUnderSampler()
    counter=np.array(range(0,len(Y_train))).reshape(-1,1)
    index,_ = rus.fit_resample(counter,Y_train[:])
    X_train = np.squeeze(X_train[index,:], axis=1)
    Y_train = np.squeeze(Y_train[index])

    print("Creating the different pipelines")
    clf = make_pipeline(CCA(n_components=2))
    print("Fitting")
    clf = clf.fit(np.array(X_train), Y_train)
    y_pred = clf.predict(X_test)
    y_pred = np.array([1 if (y >= 0.5) else 0 for y in y_pred[:,0]])
    y_test_norm = np.array([0 if y[0] == 0 else 1 for y in Y_test])
    print("getting accuracy of participant ", i)
    accuracy_adap[i] = balanced_accuracy_score(y_test_norm,y_pred)

    labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
            y_pred, dataset_moabb.codes, min_len=30, sfreq=60, consecutive=50, window_size=window_size
    )
    tps_code_adap[i] = np.mean(mean_long_accumul)
    accuracy_code_adap[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(accuracy_adap)
print(tps_code_adap)
print(accuracy_code_adap)

### Train test

In [ ]:
n_cal = 4
accuracy_code_perso = np.zeros(12)
tps_code_perso = np.zeros(12)
accuracy_perso = np.zeros(12)

for i in range(12):
    print("TL to the participant : ", i)
    X_train = raw_data[i][:int((2.2-window_size)*n_class*n_cal*60)].reshape(-1,raw_data.shape[-2]*raw_data.shape[-1])
    Y_train = labels[i][:int((2.2-window_size)*n_class*n_cal*60)]
    Y_train = np.vstack((Y_train,np.abs(1-Y_train))).T
    
    Y_test = labels[i][int((2.2-window_size)*n_class*n_cal*60):]
    X_test = raw_data[i][int((2.2-window_size)*n_class*n_cal*60):].reshape(-1,raw_data.shape[-2]*raw_data.shape[-1])
    labels_code_test = labels_code[i][n_cal*4:]
    Y_test = np.vstack((Y_test,np.abs(1-Y_test))).T

    print(X_train.shape)
    print(X_test.shape)
    X_std = X_train.std(axis=0)
    X_train /= X_std + 1e-8
    X_std = X_test.std(axis=0)
    X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    rus = RandomUnderSampler()
    counter=np.array(range(0,len(Y_train))).reshape(-1,1)
    index,_ = rus.fit_resample(counter,Y_train[:])
    X_train = np.squeeze(X_train[index,:], axis=1)
    Y_train = np.squeeze(Y_train[index])

    print("Creating the different pipelines")
    clf = make_pipeline(CCA(n_components=2))
    print("Fitting")
    clf = clf.fit(np.array(X_train), Y_train)
    y_pred = clf.predict(X_test)
    y_pred = np.array([1 if (y >= 0.5) else 0 for y in y_pred[:,0]])
    y_test_norm = np.array([0 if y[0] == 0 else 1 for y in Y_test])
    print("getting accuracy of participant ", i)
    accuracy_perso[i] = balanced_accuracy_score(y_test_norm,y_pred)

    labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
            y_pred, dataset_moabb.codes, min_len=30, sfreq=60, consecutive=50, window_size=window_size
    )
    tps_code_perso[i] = np.mean(mean_long_accumul)
    accuracy_code_perso[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(accuracy_perso)
print(tps_code_perso)
print(accuracy_code_perso)

### Plot

In [ ]:
plt.plot(accuracy_code_looa,label='looa')
plt.plot(accuracy_code_adap,label='adap')
plt.plot(accuracy_code_perso,label='perso')
plt.legend()

## Classifier : Xdawn + LDA:
1. Leave one out of all transfer learning
2. Adaptative LOOA
3. Personnal training


### Prepare data

In [31]:
subjects = [1,2,3,4,5,6,7,8,9,10,11,12]
on_frame = True
tospd = False

In [32]:
data,labels,codes,labels_codes = get_BVEP_data(subjects,on_frame)
X_parent,Y_parent,domains_parent = prepare_data(subjects,data,labels,on_frame,tospd,codes)

Choosing the first None classes from all possible events.


None
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad ep

C:\Users\s.velut\Documents\These\moabb\moabb\datasets\castillos2023.py:131: RuntimeWarning: Data file name in EEG.data (P13_burst100.fdt) is incorrect, the file name must have changed on disk, using the correct file name (P6_burst100.fdt).
  raw = mne.io.read_raw_eeglab(file_path_list[0], preload=True, verbose=False)


C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs 

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


...

0 bad epochs dropped
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 1651 samples (3.302 s)

7020 events found
Event IDs: [100 101]
Not setting metadata
7020 matching events found
Setting baseline interval to [-0.01, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 7020 events and 131 original time points ...


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


0 bad epochs dropped
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 1651 samples (3.302 s)

7020 events found
Event IDs: [100 101]
Not setting metadata
7020 matching events found
Setting baseline interval to [-0.01, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 7020 events and 131 original time points ...


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


0 bad epochs dropped


### LOO

In [22]:
XDCLDA_accuracy_code_loo = np.zeros(12)
XDCLDA_tps_code_loo = np.zeros(12)
XDCLDA_accuracy_loo = np.zeros(12)

for i in range(12):
    print("TL to the participant : ", i)
    ind2take = [j for j in range(12) if j!=i]
    X = X_parent.copy()
    Y = Y_parent.copy()
    domains = domains_parent.copy()

    X_train = X[ind2take].reshape(-1,X.shape[-2],X.shape[-1])
    Y_train = Y[ind2take].reshape(-1)
    domains_train = domains[ind2take].reshape(-1)
    X_test = X[i]
    Y_test = Y[i]
    labels_code_test = labels_codes[i]

    print(X_train.shape)
    print(X_test.shape)
    X_std = X_train.std(axis=0)
    X_train /= X_std + 1e-8
    X_std = X_test.std(axis=0)
    X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    X_train,Y_train,domains_train = balance(X_train,Y_train,domains_train)

    print("Creating the different pipelines")
    clf = make_pipeline(Xdawn(nfilter=8, estimator="lwf"),Vectorizer(),
            LDA(solver="lsqr", shrinkage="auto"))
    print("Fitting")
    clf = clf.fit(np.array(X_train), Y_train)
    y_pred = clf.predict(X_test)
    print("getting accuracy of participant ", i)
    XDCLDA_accuracy_loo[i] = balanced_accuracy_score(Y_test,y_pred)

    labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
            y_pred, codes, min_len=30, sfreq=60, consecutive=50, window_size=window_size
    )
    XDCLDA_tps_code_loo[i] = np.mean(mean_long_accumul)
    XDCLDA_accuracy_code_loo[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(XDCLDA_accuracy_loo)
print(XDCLDA_accuracy_code_loo)
print(XDCLDA_tps_code_loo)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/XDawn_LDA/score/LOO_score",XDCLDA_accuracy_loo)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/XDawn_LDA/score_code/LOO_score_code",XDCLDA_accuracy_code_loo)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/XDawn_LDA/temps_code/LOO_tps_code",XDCLDA_tps_code_loo)

TL to the participant :  0
(77220, 32, 131)
(7020, 32, 131)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  0
TL to the participant :  1
(77220, 32, 131)
(7020, 32, 131)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  1
TL to the participant :  2
(77220, 32, 131)
(7020, 32, 131)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  2
TL to the participant :  3
(77220, 32, 131)
(7020, 32, 131)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  3
TL to the participant :  4
(77220, 32, 131)
(7020, 32, 131)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  4
TL to the participant :  5
(77220, 32, 131)
(7020, 32, 131)
balancing the number of ones and zeros
Creating the different pip

### Adaptative LOO

In [24]:
n_cal = 4
n_class = 4
XDCLDA_accuracy_code_looa = np.zeros(12)
XDCLDA_tps_code_looa = np.zeros(12)
XDCLDA_accuracy_looa = np.zeros(12)

for i in range(12):
    print("TL to the participant : ", i)
    ind2take = [j for j in range(12) if j!=i]
    X = X_parent.copy()
    Y = Y_parent.copy()
    domains = domains_parent.copy()

    X_source = X[ind2take].reshape(-1,X.shape[-2],X.shape[-1])
    X_target = X[i]
    X_train = np.concatenate([X_source,X_target[:int((2.2-window_size)*n_class*n_cal*60)]],axis=0)
    Y_train = np.concatenate([Y[ind2take].reshape(-1),Y[i][:int((2.2-window_size)*n_class*n_cal*60)]],axis=0)
    domains_train = np.concatenate([domains[ind2take].reshape(-1),domains[i][:int((2.2-window_size)*n_class*n_cal*60)]],axis=0)
    
    Y_test = Y[i][int((2.2-window_size)*n_class*n_cal*60):]
    X_test = X_target[int((2.2-window_size)*n_class*n_cal*60):]
    labels_code_test = labels_codes[i][n_cal*n_class:]

    print(X_train.shape)
    print(X_test.shape)
    X_std = X_train.std(axis=0)
    X_train /= X_std + 1e-8
    X_std = X_test.std(axis=0)
    X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    X_train,Y_train,domains_train = balance(X_train,Y_train,domains_train)

    print("Creating the different pipelines")
    clf = make_pipeline(Xdawn(nfilter=8, estimator="lwf"),Vectorizer(),
            LDA(solver="lsqr", shrinkage="auto"))
    print("Fitting")
    clf = clf.fit(np.array(X_train), Y_train)
    y_pred = clf.predict(X_test)
    print("getting accuracy of participant ", i)
    XDCLDA_accuracy_looa[i] = balanced_accuracy_score(Y_test,y_pred)

    labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
            y_pred, codes, min_len=30, sfreq=60, consecutive=50, window_size=window_size
    )
    XDCLDA_tps_code_looa[i] = np.mean(mean_long_accumul)
    XDCLDA_accuracy_code_looa[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(XDCLDA_accuracy_looa)
print(XDCLDA_tps_code_looa)
print(XDCLDA_accuracy_code_looa)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/XDawn_LDA/score/LOOA_score",XDCLDA_accuracy_looa)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/XDawn_LDA/score_code/LOOA_score_code",XDCLDA_accuracy_code_looa)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/XDawn_LDA/temps_code/LOOA_tps_code",XDCLDA_tps_code_looa)

TL to the participant :  0
(79092, 32, 131)
(5148, 32, 131)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  0
TL to the participant :  1
(79092, 32, 131)
(5148, 32, 131)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  1
TL to the participant :  2
(79092, 32, 131)
(5148, 32, 131)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  2
TL to the participant :  3
(79092, 32, 131)
(5148, 32, 131)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  3
TL to the participant :  4
(79092, 32, 131)
(5148, 32, 131)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  4
TL to the participant :  5
(79092, 32, 131)
(5148, 32, 131)
balancing the number of ones and zeros
Creating the different pip

### Train test

In [33]:
n_cal = 4
n_class = 4
XDCLDA_accuracy_code_perso = np.zeros(12)
XDCLDA_tps_code_perso = np.zeros(12)
XDCLDA_accuracy_perso = np.zeros(12)

for i in range(12):
    print("TL to the participant : ", i)
    X = X_parent.copy()
    Y = Y_parent.copy()
    domains = domains_parent.copy()

    X_train = X[i][:int((2.2-window_size)*n_class*n_cal*60)]
    Y_train = Y[i][:int((2.2-window_size)*n_class*n_cal*60)]
    domains_train = domains[i][:int((2.2-window_size)*n_class*n_cal*60)]
    
    Y_test = Y[i][int((2.2-window_size)*n_class*n_cal*60):]
    X_test = X[i][int((2.2-window_size)*n_class*n_cal*60):]
    labels_code_test = labels_codes[i][n_cal*4:]

    print(X_train.shape)
    print(X_test.shape)
    X_std = X_train.std(axis=0)
    X_train /= X_std + 1e-8
    X_std = X_test.std(axis=0)
    X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    X_train,Y_train,domains_train = balance(X_train,Y_train,domains_train)

    print("Creating the different pipelines")
    clf = make_pipeline(Xdawn(nfilter=8, estimator="lwf"),Vectorizer(),
            LDA(solver="lsqr", shrinkage="auto"))
    print("Fitting")
    clf = clf.fit(np.array(X_train), Y_train)
    y_pred = clf.predict(X_test)
    print("getting accuracy of participant ", i)
    XDCLDA_accuracy_perso[i] = balanced_accuracy_score(Y_test,y_pred)

    labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
            y_pred, codes, min_len=30, sfreq=60, consecutive=50, window_size=window_size
    )
    XDCLDA_tps_code_perso[i] = np.mean(mean_long_accumul)
    XDCLDA_accuracy_code_perso[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(XDCLDA_accuracy_perso)
print(XDCLDA_tps_code_perso)
print(XDCLDA_accuracy_code_perso)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/XDawn_LDA/score/WO_score",XDCLDA_accuracy_perso)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/XDawn_LDA/score_code/WO_score_code",XDCLDA_accuracy_code_perso)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/XDawn_LDA/temps_code/WO_tps_code",XDCLDA_tps_code_perso)

TL to the participant :  0
(1872, 32, 131)
(5148, 32, 131)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  0
TL to the participant :  1
(1872, 32, 131)
(5148, 32, 131)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  1
TL to the participant :  2
(1872, 32, 131)
(5148, 32, 131)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  2
TL to the participant :  3
(1872, 32, 131)
(5148, 32, 131)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  3
TL to the participant :  4
(1872, 32, 131)
(5148, 32, 131)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  4
TL to the participant :  5
(1872, 32, 131)
(5148, 32, 131)
balancing the number of ones and zeros
Creating the different pipelines

### Plot

In [ ]:
plt.plot(accuracy_code_looa,label='looa')
plt.plot(accuracy_code_adap,label='adap')
plt.plot(accuracy_code_perso,label='perso')
plt.legend()

## Classifier : XdawnCov + TS + LDA:
1. Leave one out of all transfer learning
2. Adaptative LOOA
3. Personnal training


### LOO algo

In [ ]:
accuracy_code_looa = np.zeros(12)
tps_code_looa = np.zeros(12)
accuracy_looa = np.zeros(12)

for i in range(12):
    print("TL to the participant : ", i)
    ind2take = [j for j in range(12) if j!=i]
    X_train = raw_data[ind2take].reshape(-1,raw_data.shape[-2],raw_data.shape[-1])
    Y_train = labels[ind2take].reshape(-1)
    X_test = raw_data[i]
    Y_test = labels[i]
    labels_code_test = labels_code[i]

    print(X_train.shape)
    print(X_test.shape)
    X_std = X_train.std(axis=0)
    X_train /= X_std + 1e-8
    X_std = X_test.std(axis=0)
    X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    rus = RandomUnderSampler()
    counter=np.array(range(0,len(Y_train))).reshape(-1,1)
    index,_ = rus.fit_resample(counter,Y_train[:])
    X_train = np.squeeze(X_train[index,:,:], axis=1)
    Y_train = np.squeeze(Y_train[index])

    print("Creating the different pipelines")
    clf = make_pipeline(XdawnCovariances(nfilter=8, estimator="oas", xdawn_estimator="lwf"),
            TangentSpace(),
            LDA(solver="lsqr", shrinkage="auto"))
    
    print("Fitting")
    clf = clf.fit(np.array(X_train), Y_train)
    y_pred = clf.predict(X_test)
    print("getting accuracy of participant ", i)
    accuracy_looa[i] = balanced_accuracy_score(Y_test,y_pred)

    labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
            y_pred, dataset_moabb.codes, min_len=30, sfreq=60, consecutive=50, window_size=window_size
    )
    tps_code_looa[i] = np.mean(mean_long_accumul)
    accuracy_code_looa[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(accuracy_looa)
print(accuracy_code_looa)
print(tps_code_looa)


### LOO super trial

In [ ]:
accuracy_code_looa = np.zeros(12)
tps_code_looa = np.zeros(12)
accuracy_looa = np.zeros(12)

for i in range(12):
    print("TL to the participant : ", i)
    ind2take = [j for j in range(12) if j!=i]
    X_train = raw_data[ind2take].reshape(-1,raw_data.shape[-2],raw_data.shape[-1])
    Y_train = labels[ind2take].reshape(-1)
    
    X_test = raw_data[i]
    Y_test = labels[i]
    # Creation of super trial
    X_plus = np.mean(X_train[Y_train==1],axis=0)
    X_train = np.insert(X_train, [-1], X_plus, axis=1)
    print(np.mean(X_train[Y_train==1],axis=0).shape)
    X_test = np.insert(X_test, [-1], X_plus, axis=1)
    labels_code_test = labels_code[i]

    X_std = X_train.std(axis=0)
    X_train /= X_std + 1e-8
    X_std = X_test.std(axis=0)
    X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    rus = RandomUnderSampler()
    counter=np.array(range(0,len(Y_train))).reshape(-1,1)
    index,_ = rus.fit_resample(counter,Y_train[:])
    X_train = np.squeeze(X_train[index,:,:], axis=1)
    Y_train = np.squeeze(Y_train[index])

    print("Creating the different pipelines")
    clf = make_pipeline(XdawnCovariances(nfilter=8, estimator="oas", xdawn_estimator="lwf"),
            TangentSpace(),
            LDA(solver="lsqr", shrinkage="auto"))
    
    print("Fitting")
    clf = clf.fit(np.array(X_train), Y_train)
    y_pred = clf.predict(X_test)
    print("getting accuracy of participant ", i)
    accuracy_looa[i] = balanced_accuracy_score(Y_test,y_pred)

    labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
            y_pred, dataset_moabb.codes, min_len=30, sfreq=60, consecutive=50, window_size=window_size
    )
    tps_code_looa[i] = np.mean(mean_long_accumul)
    accuracy_code_looa[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()
    print("score accuracy:", accuracy_looa[i])
    print("score code accuracy:", accuracy_code_looa[i])

print(accuracy_looa)
print(accuracy_code_looa)
print(tps_code_looa)


### adaptative LOO

In [ ]:
n_cal = 4
accuracy_code_adap = np.zeros(12)
tps_code_adap = np.zeros(12)
accuracy_adap = np.zeros(12)

for i in range(12):
    print("TL to the participant : ", i)
    ind2take = [j for j in range(12) if j!=i]
    X_source = raw_data[ind2take].reshape(-1,raw_data.shape[-2],raw_data.shape[-1])
    X_target = raw_data[i]
    X_train = np.concatenate([X_source,X_target[:int((2.2-window_size)*n_class*n_cal*60)]],axis=0)
    Y_train = np.concatenate([labels[ind2take].reshape(-1),labels[i][:int((2.2-window_size)*n_class*n_cal*60)]],axis=0)
    
    Y_test = labels[i][int((2.2-window_size)*n_class*n_cal*60):]
    X_test = X_target[int((2.2-window_size)*n_class*n_cal*60):]
    labels_code_test = labels_code[i][n_cal*4:]

    print(X_train.shape)
    print(X_test.shape)
    X_std = X_train.std(axis=0)
    X_train /= X_std + 1e-8
    X_std = X_test.std(axis=0)
    X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    rus = RandomUnderSampler()
    counter=np.array(range(0,len(Y_train))).reshape(-1,1)
    index,_ = rus.fit_resample(counter,Y_train[:])
    X_train = np.squeeze(X_train[index,:,:], axis=1)
    Y_train = np.squeeze(Y_train[index])

    print("Creating the different pipelines")
    clf = make_pipeline(XdawnCovariances(nfilter=8, estimator="oas", xdawn_estimator="lwf"),
            TangentSpace(),
            LDA(solver="lsqr", shrinkage="auto"))
    
    print("Fitting")
    clf = clf.fit(np.array(X_train), Y_train)
    y_pred = clf.predict(X_test)
    print("getting accuracy of participant ", i)
    accuracy_adap[i] = balanced_accuracy_score(Y_test,y_pred)

    labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
            y_pred, dataset_moabb.codes, min_len=30, sfreq=60, consecutive=50, window_size=window_size
    )
    tps_code_adap[i] = np.mean(mean_long_accumul)
    accuracy_code_adap[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(accuracy_adap)
print(tps_code_adap)
print(accuracy_code_adap)

### Train test

### PLot results

In [ ]:
plt.plot(accuracy_code_looa,label='looa')
plt.plot(accuracy_code_adap,label='adap')
plt.plot(accuracy_code_perso,label='perso')
plt.legend()

## Use the CNN from Toulouse

### Prepare data

In [ ]:
raw_data_dl = []
labels_dl = []
n_channels = 32

for i in range(1,13):
    path = '/'.join(['C:\\Users\\s.velut\\Documents\\These\\Protheus_PHD\\Class4', 'P'+str(i)])
    file_name = '_'.join(['P'+str(i), 'burst100.set'])
    raw_i = mne.io.read_raw_eeglab(os.path.join(path, file_name), preload=True, verbose=False)
    raw_i = raw_i.filter(l_freq=50.1, h_freq=49.9, method="iir", verbose=True)
    mne.set_eeg_reference(raw_i, 'average', copy=False, verbose=False)

    events, event_id = mne.events_from_annotations(raw_i, event_id='auto', verbose=False)
    to_remove = []
    for idx in range(len(raw_i.annotations.description)):
        if (('collects' in raw_i.annotations.description[idx]) or
            ('iti' in raw_i.annotations.description[idx]) or
            (raw_i.annotations.description[idx] == '[]')):
            to_remove.append(idx)
        else:
            code = raw_i.annotations.description[idx].split('_')[0]
            lab = raw_i.annotations.description[idx].split('_')[1]
            code = code.replace('\n', '')
            code = code.replace('[', '')
            code = code.replace(']', '')
            code = code.replace(' ', '')
            raw_i.annotations.description[idx] = code + '_' + lab

    to_remove = np.array(to_remove)
    if len(to_remove) > 0:
        raw_i.annotations.delete(to_remove)
    # Get the events
    events, event_id = mne.events_from_annotations(raw_i, event_id='auto', verbose=False)
    shift = 0.0
    # Epoch the data following event
    epochs = mne.Epochs(raw_i, events, event_id=event_id, tmin=shift, \
                tmax=2.2+shift, baseline=(None, None), preload=False, verbose=False)
    label = epochs.events[..., -1]
    label -= np.min(label)
    labels_dl.append(label)
    data = epochs.get_data()
    info_ep = epochs.info

    raw_data_dl.append(data)

raw_data_dl = np.array(raw_data_dl)
labels_dl = np.array(labels_dl)

In [ ]:
def code2array(code):
    tmp = []
    for idx, c in enumerate(code[:-2]):
        if c == '5' or c == '.':
            continue
        elif c == '0':
            if code[idx+2] == '5':
                tmp.append(0.5)
            else:
                tmp.append(0)
        else:
            tmp.append(1)
    if code[-1] == '.':
        if code[-2] == '0':
            tmp.append(0)
        else:
            tmp.append(1)
    return np.array(tmp)

In [ ]:
codes = OrderedDict()
for k, v in event_id.items():
    code = k.split('_')[0]
    code = code.replace('.','').replace('2','')
    idx = k.split('_')[1]
    if 'randomslowwhite' in file_name:
        codes[v-1] = code2array(code) 
    else:
        codes[v-1] = np.array(list(map(int, code)))

In [ ]:
def to_window_old(data, labels):
    length = int((2.2-window_size)*sfreq)
    X = np.empty(shape=((length)*data.shape[0], n_channels, n_samples_windows))
    y = np.empty(shape=((length)*data.shape[0]), dtype=int)
    print(length)
    print(n_samples_windows)
    count = 0
    for trial_nb, trial in enumerate(data):
        lab = labels[trial_nb]
        c = codes[lab]
        code_pos = 0
        for idx in range(length):
            X[count] = trial[:, idx:idx+n_samples_windows]
            if idx/sfreq >= (code_pos+1)/fps:
                code_pos += 1 
            y[count] = int(c[code_pos])
            count += 1

    X = np.expand_dims(X, 1)
    X = X.astype(np.float32)
    y = np.vstack((y,np.abs(1-y))).T
    return X, y

### Prepare data 2

In [27]:
subjects = [1,2,3,4,5,6,7,8,9,10,11,12]
n_channels = 32
on_frame = True
tospd = False
if on_frame:
    freq = fps
else:
    freq = sfreq

In [28]:
raw_data,labels,codes,labels_codes = get_BVEP_data(subjects,on_frame)
X_parent,Y_parent,domains_parent = prepare_data(subjects,raw_data,labels,on_frame,tospd,codes)

Choosing the first None classes from all possible events.


None
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad ep

C:\Users\s.velut\Documents\These\moabb\moabb\datasets\castillos2023.py:131: RuntimeWarning: Data file name in EEG.data (P13_burst100.fdt) is incorrect, the file name must have changed on disk, using the correct file name (P6_burst100.fdt).
  raw = mne.io.read_raw_eeglab(file_path_list[0], preload=True, verbose=False)


C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs 

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


...

0 bad epochs dropped
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 1651 samples (3.302 s)

7020 events found
Event IDs: [100 101]
Not setting metadata
7020 matching events found
Setting baseline interval to [-0.01, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 7020 events and 131 original time points ...


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


0 bad epochs dropped
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 1651 samples (3.302 s)

7020 events found
Event IDs: [100 101]
Not setting metadata
7020 matching events found
Setting baseline interval to [-0.01, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 7020 events and 131 original time points ...


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


0 bad epochs dropped


In [29]:
def expand_dim(X):
    X_temp = []
    Y_temp = []
    for i in range(X.shape[0]):
        # print(i)
        X_temp.append(np.expand_dims(X[i],1))
    return np.array(X_temp)


### LOO

In [16]:
keras.backend.clear_session()
CNN_accuracy_code_loo = np.zeros(12)
CNN_tps_code_loo = np.zeros(12)
CNN_accuracy_loo = np.zeros(12)
freq = 60

for i in range(12):
    print("TL to the participant : ", i)
    ind2take = [j for j in range(12) if j!=i]
    X = X_parent.copy()
    Y = Y_parent.copy()
    domains = domains_parent.copy()
    
    X_train = np.concatenate(expand_dim(X[ind2take])).reshape(-1,1,X.shape[-2],X.shape[-1])
    Y_train = np.concatenate(Y[ind2take]).reshape(-1)
    Y_train = np.vstack((Y_train,np.abs(1-Y_train))).T
    domains_train = domains[ind2take].reshape(-1)
    X_test = np.expand_dims(X[i],1)
    Y_test = np.vstack((Y[i],np.abs(1-Y[i]))).T
    labels_code_test = labels_codes[i]

    print(X_train.shape)
    print(X_test.shape)
    X_std = X_train.std(axis=0)
    X_train /= X_std + 1e-8
    X_std = X_test.std(axis=0)
    X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    X_train,Y_train,domains_train = balance(X_train,Y_train,domains_train)
    n_samples_windows = X_train.shape[-1]

    print("Creating the different pipelines")
    clf = basearchi(windows_size = n_samples_windows, n_channel_input = n_channels)
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42, shuffle=True)
    batchsize = 64 #128 # 64 for burst
    epochs = 20 #45 # 20 for burst

    print("Fitting")
    lr = 1e-3
    weight_decay = 1e-4
    optimizer = keras.optimizers.Adam(learning_rate=lr, amsgrad=True)
    clf.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])
    history = clf.fit(np.array(x_train), y_train,
                    batch_size=batchsize, epochs=epochs,
                    validation_data=(np.array(x_val), y_val), shuffle=True)
    y_pred = clf.predict(X_test)[:,0]
    y_pred = np.array(y_pred)
    y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
    y_test_norm = np.array([0 if y[0] == 0 else 1 for y in Y_test])

    tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
    CNN_accuracy_loo[i] = balanced_accuracy_score(y_test_norm,y_pred_norm)

    print("getting accuracy of participant ", i)

    labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
        y_pred_norm, codes, min_len=30, sfreq=freq, consecutive=50, window_size=window_size
    )
    CNN_tps_code_loo[i] = np.mean(mean_long_accumul)
    CNN_accuracy_code_loo[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(CNN_accuracy_loo)
print(CNN_accuracy_code_loo)
print(CNN_tps_code_loo)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/CNN/score/LOOsample_score",CNN_accuracy_loo)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/CNN/score_code/LOOsample_score_code",CNN_accuracy_code_loo)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/CNN/temps_code/LOOsample_tps_code",CNN_tps_code_loo)


TL to the participant :  0
(77220, 1, 32, 131)
(7020, 1, 32, 131)
balancing the number of ones and zeros
Creating the different pipelines

Fitting
Epoch 1/20


372/372 [==============================] - 5s 9ms/step - loss: 0.6248 - accuracy: 0.6509 - val_loss: 0.4799 - val_accuracy: 0.7788
Epoch 2/20
372/372 [==============================] - 3s 9ms/step - loss: 0.5138 - accuracy: 0.7535 - val_loss: 0.4603 - val_accuracy: 0.7920
Epoch 3/20
372/372 [==============================] - 3s 9ms/step - loss: 0.4857 - accuracy: 0.7699 - val_loss: 0.4459 - val_accuracy: 0.7966
Epoch 4/20
372/372 [==============================] - 4s 9ms/step - loss: 0.4744 - accuracy: 0.7797 - val_loss: 0.4411 - val_accuracy: 0.8015
Epoch 5/20
372/372 [==============================] - 4s 10ms/step - loss: 0.4694 - accuracy: 0.7798 - val_loss: 0.4391 - val_accuracy: 0.8049
Epoch 6/20
372/372 [==============================] - 4s 10ms/step - loss: 0.4651 - accuracy: 0.7829 - val_loss: 0.4317 - val_accuracy: 0.80

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  11
(77220, 1, 32, 131)
(7020, 1, 32, 131)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1/20
372/372 [==============================] - 4s 9ms/step - loss: 0.5978 - accuracy: 0.6832 - val_loss: 0.4739 - val_accuracy: 0.7905
Epoch 2/20
372/372 [==============================] - 3s 9ms/step - loss: 0.4986 - accuracy: 0.7666 - val_loss: 0.4340 - val_accuracy: 0.8068
Epoch 3/20
372/372 [==============================] - 4s 11ms/step - loss: 0.4703 - accuracy: 0.7834 - val_loss: 0.4197 - val_accuracy: 0.8212
Epoch 4/20
372/372 [==============================] - 3s 9ms/step - loss: 0.4577 - accuracy: 0.7891 - val_loss: 0.4080 - val_accuracy: 0.8220
Epoch 5/20
372/372 [==============================] - 4s 10ms/step - loss: 0.4474 - accuracy: 0.7964 - val_loss: 0.4053 - val_accuracy: 0.8254
Epoch 6/20
372/372 [==============================] - 4s 10ms/step - loss: 0.4428 - accuracy: 0.7975 - val_loss: 0.3994 - val_accuracy: 0.823

### LOOA

In [18]:
keras.backend.clear_session()
n_cal = 4
n_class = 4 
CNN_accuracy_code_looa = np.zeros(12)
CNN_tps_code_looa = np.zeros(12)
CNN_accuracy_looa = np.zeros(12)
nb_samples_windows = int((2.2-window_size)*n_class*n_cal*freq)


for i in range(12):
    print("TL to the participant : ", i)
    ind2take = [j for j in range(12) if j!=i]
    X = X_parent.copy()
    Y = Y_parent.copy()
    domains = domains_parent.copy()

    X_train = np.expand_dims(np.concatenate([X[ind2take].reshape(-1,X.shape[-2],X.shape[-1]),X[i][:nb_samples_windows]]).reshape(-1,X.shape[-2],X.shape[-1]),1)
    Y_train = np.concatenate([Y[ind2take].reshape(-1),Y[i][:nb_samples_windows]]).reshape(-1)
    Y_train = np.vstack((Y_train,np.abs(1-Y_train))).T
    domains_train = np.concatenate([domains[ind2take].reshape(-1),domains[i][:nb_samples_windows]]).reshape(-1)
    X_test = np.expand_dims(X[i][nb_samples_windows:],1)
    Y_test = np.vstack((Y[i][nb_samples_windows:],np.abs(1-Y[i][nb_samples_windows:]))).T
    labels_code_test = labels_codes[i][n_cal*n_class:]

    print(X_train.shape)
    print(X_test.shape)
    X_std = X_train.std(axis=0)
    X_train /= X_std + 1e-8
    X_std = X_test.std(axis=0)
    X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    X_train,Y_train,domains_train = balance(X_train,Y_train,domains_train)
    n_samples_windows = X_train.shape[-1]

    print("Creating the different pipelines")
    clf = basearchi(windows_size = n_samples_windows, n_channel_input = n_channels)
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42, shuffle=True)
    batchsize = 64 #128 # 64 for burst
    epochs = 20 #45 # 20 for burst

    print("Fitting")
    lr = 1e-3
    weight_decay = 1e-4
    optimizer = keras.optimizers.Adam(learning_rate=lr, amsgrad=True)
    clf.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])
    history = clf.fit(np.array(x_train), y_train,
                    batch_size=batchsize, epochs=epochs,
                    validation_data=(np.array(x_val), y_val), shuffle=True)
    y_pred = clf.predict(X_test)[:,0]
    y_pred = np.array(y_pred)
    y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
    y_test_norm = np.array([0 if y[0] == 0 else 1 for y in Y_test])

    tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
    CNN_accuracy_looa[i] = balanced_accuracy_score(y_test_norm,y_pred_norm)

    print("getting accuracy of participant ", i)

    labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
        y_pred_norm, codes, min_len=30, sfreq=freq, consecutive=50, window_size=window_size
    )
    CNN_tps_code_looa[i] = np.mean(mean_long_accumul)
    CNN_accuracy_code_looa[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(CNN_accuracy_looa)
print(CNN_accuracy_code_looa)
print(CNN_tps_code_looa)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/CNN/score/LOOAsample_score",CNN_accuracy_looa)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/CNN/score_code/LOOAsample_score_code",CNN_accuracy_code_looa)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/CNN/temps_code/LOOAsample_tps_code",CNN_tps_code_looa)


TL to the participant :  0
(79092, 1, 32, 131)
(5148, 1, 32, 131)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1/20
381/381 [==============================] - 5s 9ms/step - loss: 0.6134 - accuracy: 0.6655 - val_loss: 0.5008 - val_accuracy: 0.7596
Epoch 2/20
381/381 [==============================] - 3s 9ms/step - loss: 0.5085 - accuracy: 0.7579 - val_loss: 0.4636 - val_accuracy: 0.7788
Epoch 3/20
381/381 [==============================] - 4s 10ms/step - loss: 0.4847 - accuracy: 0.7729 - val_loss: 0.4539 - val_accuracy: 0.7892
Epoch 4/20
381/381 [==============================] - 4s 10ms/step - loss: 0.4747 - accuracy: 0.7812 - val_loss: 0.4475 - val_accuracy: 0.8025
Epoch 5/20
381/381 [==============================] - 4s 11ms/step - loss: 0.4656 - accuracy: 0.7860 - val_loss: 0.4384 - val_accuracy: 0.7992
Epoch 6/20
381/381 [==============================] - 4s 11ms/step - loss: 0.4619 - accuracy: 0.7871 - val_loss: 0.4377 - val_accuracy: 0.803

### Train Test

In [30]:
n_cal = 7
n_class = 4
CNN_accuracy_code_perso = np.zeros(12)
CNN_tps_code_perso = np.zeros(12)
CNN_accuracy_perso = np.zeros(12)
nb_samples_windows = int((2.2-window_size)*n_class*n_cal*freq)


for i in range(12):
    print("TL to the participant : ", i)
    X = X_parent.copy()
    Y = Y_parent.copy()
    domains = domains_parent.copy()

    X_train = np.expand_dims(X[i][:nb_samples_windows],1)
    Y_train = Y[i][:nb_samples_windows]
    Y_train = np.vstack((Y_train,np.abs(1-Y_train))).T
    domains_train = domains[i][:nb_samples_windows]
    X_test = np.expand_dims(X[i][nb_samples_windows:],1)
    Y_test = np.vstack((Y[i][nb_samples_windows:],np.abs(1-Y[i][nb_samples_windows:]))).T
    labels_code_test = labels_codes[i][n_cal*n_class:]

    print(X_train.shape)

    print(X_train.shape)
    print(X_test.shape)
    X_std = X_train.std(axis=0)
    X_train /= X_std + 1e-8
    X_std = X_test.std(axis=0)
    X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    X_train,Y_train,domains_train = balance(X_train,Y_train,domains_train)
    n_samples_windows = X_train.shape[-1]

    print("Creating the different pipelines")
    clf = basearchi(windows_size = n_samples_windows, n_channel_input = X_train.shape[2])
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42, shuffle=True)
    batchsize = 64 #128 # 64 for burst
    epochs = 20 #45 # 20 for burst

    print("Fitting")
    lr = 1e-3
    weight_decay = 1e-4
    optimizer = keras.optimizers.Adam(learning_rate=lr, amsgrad=True)
    clf.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])
    history = clf.fit(np.array(x_train), y_train,
                    batch_size=batchsize, epochs=epochs,
                    validation_data=(np.array(x_val), y_val), shuffle=True)

    print("getting accuracy of participant ", i)
    y_pred = clf.predict(X_test)[:,0]
    y_pred = np.array(y_pred)
    y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
    y_test_norm = np.array([0 if y[0] == 0 else 1 for y in Y_test])

    tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
    CNN_accuracy_perso[i] = balanced_accuracy_score(y_test_norm,y_pred_norm)

    labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
        y_pred_norm, codes, min_len=30, sfreq=freq, consecutive=50, window_size=window_size
    )
    CNN_tps_code_perso[i] = np.mean(mean_long_accumul)
    CNN_accuracy_code_perso[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(CNN_accuracy_perso)
print(CNN_tps_code_perso)
print(CNN_accuracy_code_perso)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/CNN/score/WOsample_score",CNN_accuracy_perso)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/CNN/score_code/WOsample_score_code",CNN_accuracy_code_perso)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/CNN/temps_code/WOsample_tps_code",CNN_tps_code_perso)


TL to the participant :  0
(3276, 1, 32, 131)
(3276, 1, 32, 131)
(3744, 1, 32, 131)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1/20
16/16 [==============================] - 4s 28ms/step - loss: 0.7648 - accuracy: 0.5228 - val_loss: 0.6936 - val_accuracy: 0.6071
Epoch 2/20
16/16 [==============================] - 0s 13ms/step - loss: 0.7224 - accuracy: 0.5615 - val_loss: 0.6780 - val_accuracy: 0.6161
Epoch 3/20
16/16 [==============================] - 0s 13ms/step - loss: 0.6993 - accuracy: 0.5665 - val_loss: 0.6587 - val_accuracy: 0.6250
Epoch 4/20
16/16 [==============================] - 0s 15ms/step - loss: 0.6445 - accuracy: 0.6458 - val_loss: 0.6345 - val_accuracy: 0.6518
Epoch 5/20
16/16 [==============================] - 0s 13ms/step - loss: 0.6224 - accuracy: 0.6696 - val_loss: 0.5947 - val_accuracy: 0.7054
Epoch 6/20
16/16 [==============================] - 0s 14ms/step - loss: 0.5929 - accuracy: 0.6905 - val_loss: 0.5644 - val_accurac

### Plot

In [ ]:
# plt.plot(accuracy_code_looa,label='looa')
# plt.plot(accuracy_code_adap,label='adap')
plt.plot(accuracy_code_perso,label='perso')
plt.legend()

## Test on the SPDNet

### Prepare the data

In [ ]:
raw_data_dl = []
labels_dl = []
n_channels = 32

for i in range(1,13):
    path = '/'.join(['C:\\Users\\s.velut\\Documents\\These\\Protheus_PHD\\Class4', 'P'+str(i)])
    file_name = '_'.join(['P'+str(i), 'burst100.set'])
    raw_i = mne.io.read_raw_eeglab(os.path.join(path, file_name), preload=True, verbose=False)
    raw_i = raw_i.filter(l_freq=50.1, h_freq=49.9, method="iir", verbose=True)
    mne.set_eeg_reference(raw_i, 'average', copy=False, verbose=False)

    events, event_id = mne.events_from_annotations(raw_i, event_id='auto', verbose=False)
    to_remove = []
    for idx in range(len(raw_i.annotations.description)):
        if (('collects' in raw_i.annotations.description[idx]) or
            ('iti' in raw_i.annotations.description[idx]) or
            (raw_i.annotations.description[idx] == '[]')):
            to_remove.append(idx)
        else:
            code = raw_i.annotations.description[idx].split('_')[0]
            lab = raw_i.annotations.description[idx].split('_')[1]
            code = code.replace('\n', '')
            code = code.replace('[', '')
            code = code.replace(']', '')
            code = code.replace(' ', '')
            raw_i.annotations.description[idx] = code + '_' + lab

    to_remove = np.array(to_remove)
    if len(to_remove) > 0:
        raw_i.annotations.delete(to_remove)
    # Get the events
    events, event_id = mne.events_from_annotations(raw_i, event_id='auto', verbose=False)
    shift = 0.0
    # Epoch the data following event
    epochs = mne.Epochs(raw_i, events, event_id=event_id, tmin=shift, \
                tmax=2.2+shift, baseline=(None, None), preload=False, verbose=False)
    label = epochs.events[..., -1]
    label -= np.min(label)
    labels_dl.append(label)
    data = epochs.get_data()
    info_ep = epochs.info

    raw_data_dl.append(data)

raw_data_dl = np.array(raw_data_dl)
labels_dl = np.array(labels_dl)

In [ ]:
def code2array(code):
    tmp = []
    for idx, c in enumerate(code[:-2]):
        if c == '5' or c == '.':
            continue
        elif c == '0':
            if code[idx+2] == '5':
                tmp.append(0.5)
            else:
                tmp.append(0)
        else:
            tmp.append(1)
    if code[-1] == '.':
        if code[-2] == '0':
            tmp.append(0)
        else:
            tmp.append(1)
    return np.array(tmp)

In [ ]:
from collections import OrderedDict
codes = OrderedDict()
for k, v in event_id.items():
    code = k.split('_')[0]
    code = code.replace('.','').replace('2','')
    idx = k.split('_')[1]
    if 'randomslowwhite' in file_name:
        codes[v-1] = code2array(code) 
    else:
        codes[v-1] = np.array(list(map(int, code)))

In [ ]:
def to_window_cov(data, labels):
    n_samples_windows = int(window_size*sfreq)
    length = int((2.2-window_size)*sfreq)
    X = np.empty(shape=((length)*data.shape[0], n_channels, n_samples_windows))
    y = np.empty(shape=((length)*data.shape[0]), dtype=int)
    print(length)
    print(n_samples_windows)
    count = 0
    for trial_nb, trial in enumerate(data):
        lab = labels[trial_nb]
        c = codes[lab]
        code_pos = 0
        for idx in range(length):
            X[count] = trial[:, idx:idx+n_samples_windows]
            if idx/sfreq >= (code_pos+1)/fps:
                code_pos += 1 
            y[count] = int(c[code_pos])
            count += 1

    xdawncov = XdawnCovariances(estimator="lwf",xdawn_estimator="lwf",nfilter=8)
    X = xdawncov.fit_transform(X,y)
    # X = X.astype(np.float32)
    y_pred = np.vstack((y,np.abs(1-y))).T
    y = np.array([1 if (y >= 0.5) else 0 for y in y_pred[:,0]])
    return X, y

In [ ]:
def data_train_by_parti(ind2take,raw_data_dl,labels_dl):
    X_train = []
    Y_train = []
    for k in ind2take:
        temp_X, temp_Y = to_window_cov(raw_data_dl[k],labels_dl[k])
        X_train.append(temp_X)
        Y_train.append(temp_Y)
    return np.array(X_train), np.array(Y_train)

### Prepare data 2

In [6]:
subjects = [1,2,3,4,5,6,7,8,9,10,11,12]
n_channels = 32
on_frame = True
if on_frame:
    freq = fps
else:
    freq = sfreq

In [7]:
raw_data,labels,codes,labels_codes = get_BVEP_data(subjects,on_frame)
X_parent,Y_parent,domains_parent = prepare_data(subjects,raw_data,labels,on_frame,True,codes)

Choosing the first None classes from all possible events.


None
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad ep

C:\Users\s.velut\Documents\These\moabb\moabb\datasets\castillos2023.py:131: RuntimeWarning: Data file name in EEG.data (P13_burst100.fdt) is incorrect, the file name must have changed on disk, using the correct file name (P6_burst100.fdt).
  raw = mne.io.read_raw_eeglab(file_path_list[0], preload=True, verbose=False)


C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
C:\Users\s.velut\mne_data\MNE-4class-vep-data\records\8255618\files\
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs 

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


...

0 bad epochs dropped
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 1651 samples (3.302 s)

7020 events found
Event IDs: [100 101]
Not setting metadata
7020 matching events found
Setting baseline interval to [-0.01, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 7020 events and 131 original time points ...


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


0 bad epochs dropped
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 1651 samples (3.302 s)

7020 events found
Event IDs: [100 101]
Not setting metadata
7020 matching events found
Setting baseline interval to [-0.01, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 7020 events and 131 original time points ...


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


0 bad epochs dropped


In [7]:
X_parent.shape

(12, 7020, 32, 32)

### LOO

In [11]:
keras.backend.clear_session()
spd_accuracy_code_loo = np.zeros(12)
spd_tps_code_loo = np.zeros(12)
spd_accuracy_loo = np.zeros(12)
batchsize = 64 #128 # 64 for burst
epoch = 20 #45 # 20 for burst

for i in range(12):
    print("TL to the participant : ", i)
    ind2take = [j for j in range(12) if j!=i]
    X = X_parent.copy()
    Y = Y_parent.copy()
    domains = domains_parent.copy()

    X_train = np.concatenate(X[ind2take]).reshape(-1,X.shape[-2],X.shape[-1])
    Y_train = np.concatenate(Y[ind2take]).reshape(-1)
    X_test = X[i]
    Y_test = Y[i]
    # X_test = xdawncov.transform(X_test)

    labels_code_test = labels_codes[i]

    print(X_train.shape)
    print(X_test.shape)
    X_std = X_train.std(axis=0)
    X_train /= X_std + 1e-8
    X_std = X_test.std(axis=0)
    X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    rus = RandomUnderSampler()
    counter=np.array(range(0,len(Y_train))).reshape(-1,1)
    index,_ = rus.fit_resample(counter,Y_train[:])
    X_train = np.squeeze(X_train[index,:,:], axis=1)
    Y_train = np.squeeze(Y_train[index])

    print("Creating the different pipelines")
    lr = 1e-3
    # optimizer = riemannian_adam.RiemannianAdam(learning_rate=lr)
    # clf = SPDNet_AJD(n_epochs=epoch,batch_size=batchsize,valid_split=0.1)
    clf = SPDNet_Tensorflow(bimap_dims=[28,14,7])

    print("Fitting")
    weight_decay = 1e-4
    
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42, shuffle=True)
    history = clf.fit(np.array(x_train), y_train,
                    batch_size=batchsize, epochs=epoch,
                    validation_data=(np.array(x_val), y_val), shuffle=True)
    
    print("getting accuracy of participant ", i)
    y_pred = clf.predict(X_test)[:]
    y_pred = np.array(y_pred)
    y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
    y_test_norm = np.array([0 if y == 0 else 1 for y in Y_test])

    tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
    spd_accuracy_loo[i] = (tp+tn)/len(y_test_norm)

    labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
        y_pred_norm, codes, min_len=30, sfreq=fps, consecutive=50, window_size=window_size
    )
    spd_tps_code_loo[i] = np.mean(mean_long_accumul)
    spd_accuracy_code_loo[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    
    keras.backend.clear_session()
    del X_train
    del X_test
    del Y_train
    del Y_test

print(spd_accuracy_loo)
print(spd_accuracy_code_loo)
print(spd_tps_code_loo)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDNet/LOO_score",spd_accuracy_loo)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDNet/LOO_score_code",spd_accuracy_code_loo)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDNet/LOO_tps_code",spd_tps_code_loo)



TL to the participant :  0
(77220, 32, 32)
(7020, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting

Epoch 1/20

372/372 [==============================] - 5s 12ms/step - loss: 0.5636 - sparse_categorical_accuracy: 0.7420 - val_loss: 0.4546 - val_sparse_categorical_accuracy: 0.7909
Epoch 2/20
372/372 [==============================] - 4s 11ms/step - loss: 0.4159 - sparse_categorical_accuracy: 0.8186 - val_loss: 0.4028 - val_sparse_categorical_accuracy: 0.8189
Epoch 3/20
372/372 [==============================] - 4s 12ms/step - loss: 0.4022 - sparse_categorical_accuracy: 0.8218 - val_loss: 0.3987 - val_sparse_categorical_accuracy: 0.8186
Epoch 4/20
372/372 [==============================] - 5s 13ms/step - loss: 0.3998 - sparse_categorical_accuracy: 0.8225 - val_loss: 0.3887 - val_sparse_categorical_accuracy: 0.8273
Epoch 5/20
372/372 [==============================] - 5s 13ms/step - loss: 0.3972 - sparse_categorical_accuracy: 0.8223 - val_loss: 0.4

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  1
(77220, 32, 32)
(7020, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1/20
372/372 [==============================] - 5s 12ms/step - loss: 0.6493 - sparse_categorical_accuracy: 0.7119 - val_loss: 0.5012 - val_sparse_categorical_accuracy: 0.7553
Epoch 2/20
372/372 [==============================] - 4s 12ms/step - loss: 0.4206 - sparse_categorical_accuracy: 0.8202 - val_loss: 0.4043 - val_sparse_categorical_accuracy: 0.8205
Epoch 3/20
372/372 [==============================] - 5s 12ms/step - loss: 0.3916 - sparse_categorical_accuracy: 0.8262 - val_loss: 0.3997 - val_sparse_categorical_accuracy: 0.8269
Epoch 4/20
372/372 [==============================] - 4s 12ms/step - loss: 0.3843 - sparse_categorical_accuracy: 0.8293 - val_loss: 0.4007 - val_sparse_categorical_accuracy: 0.8189
Epoch 5/20
372/372 [==============================] - 4s 11ms/step - loss: 0.3814 - sparse_categorical_accuracy: 0.8322 - val_loss: 0.3854

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  4
(77220, 32, 32)
(7020, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1/20
372/372 [==============================] - 5s 12ms/step - loss: 0.6599 - sparse_categorical_accuracy: 0.7054 - val_loss: 0.4487 - val_sparse_categorical_accuracy: 0.8155
Epoch 2/20
372/372 [==============================] - 5s 12ms/step - loss: 0.4204 - sparse_categorical_accuracy: 0.8184 - val_loss: 0.4012 - val_sparse_categorical_accuracy: 0.8254
Epoch 3/20
372/372 [==============================] - 5s 13ms/step - loss: 0.4017 - sparse_categorical_accuracy: 0.8209 - val_loss: 0.4128 - val_sparse_categorical_accuracy: 0.8155
Epoch 4/20
372/372 [==============================] - 5s 12ms/step - loss: 0.3960 - sparse_categorical_accuracy: 0.8250 - val_loss: 0.3906 - val_sparse_categorical_accuracy: 0.8246
Epoch 5/20
372/372 [==============================] - 5s 12ms/step - loss: 0.3928 - sparse_categorical_accuracy: 0.8251 - val_loss: 0.4313

...

TL to the participant :  7
(77220, 32, 32)
(7020, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1/20
372/372 [==============================] - 6s 14ms/step - loss: 0.6883 - sparse_categorical_accuracy: 0.6868 - val_loss: 0.4797 - val_sparse_categorical_accuracy: 0.7996
Epoch 2/20
372/372 [==============================] - 5s 14ms/step - loss: 0.4222 - sparse_categorical_accuracy: 0.8159 - val_loss: 0.4301 - val_sparse_categorical_accuracy: 0.8019
Epoch 3/20
372/372 [==============================] - 5s 14ms/step - loss: 0.4023 - sparse_categorical_accuracy: 0.8196 - val_loss: 0.4253 - val_sparse_categorical_accuracy: 0.8042
Epoch 4/20
372/372 [==============================] - 5s 14ms/step - loss: 0.3985 - sparse_categorical_accuracy: 0.8218 - val_loss: 0.4213 - val_sparse_categorical_accuracy: 0.8057
Epoch 5/20
372/372 [==============================] - 5s 14ms/step - loss: 0.3953 - sparse_categorical_accuracy: 0.8228 - val_loss: 0.4186

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  9
(77220, 32, 32)
(7020, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1/20
372/372 [==============================] - 6s 14ms/step - loss: 0.5803 - sparse_categorical_accuracy: 0.7429 - val_loss: 0.4572 - val_sparse_categorical_accuracy: 0.7856
Epoch 2/20
372/372 [==============================] - 5s 14ms/step - loss: 0.4235 - sparse_categorical_accuracy: 0.8099 - val_loss: 0.4218 - val_sparse_categorical_accuracy: 0.8091
Epoch 3/20
372/372 [==============================] - 5s 15ms/step - loss: 0.4127 - sparse_categorical_accuracy: 0.8168 - val_loss: 0.4182 - val_sparse_categorical_accuracy: 0.8095
Epoch 4/20
372/372 [==============================] - 5s 15ms/step - loss: 0.4096 - sparse_categorical_accuracy: 0.8160 - val_loss: 0.4209 - val_sparse_categorical_accuracy: 0.8091
Epoch 5/20
372/372 [==============================] - 5s 14ms/step - loss: 0.4068 - sparse_categorical_accuracy: 0.8171 - val_loss: 0.4214

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  11
(77220, 32, 32)
(7020, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1/20
372/372 [==============================] - 7s 16ms/step - loss: 0.6316 - sparse_categorical_accuracy: 0.7168 - val_loss: 0.4517 - val_sparse_categorical_accuracy: 0.8049
Epoch 2/20
372/372 [==============================] - 6s 15ms/step - loss: 0.4025 - sparse_categorical_accuracy: 0.8287 - val_loss: 0.4066 - val_sparse_categorical_accuracy: 0.8193
Epoch 3/20
372/372 [==============================] - 6s 15ms/step - loss: 0.3833 - sparse_categorical_accuracy: 0.8314 - val_loss: 0.4176 - val_sparse_categorical_accuracy: 0.8140
Epoch 4/20
372/372 [==============================] - 5s 15ms/step - loss: 0.3803 - sparse_categorical_accuracy: 0.8321 - val_loss: 0.4377 - val_sparse_categorical_accuracy: 0.8034
Epoch 5/20
372/372 [==============================] - 6s 15ms/step - loss: 0.3779 - sparse_categorical_accuracy: 0.8337 - val_loss: 0.392

### LOOA adaptative

In [12]:
keras.backend.clear_session()
window_size=0.25
n_cal = 1
n_class = 4
nb_samples_cal = int(n_class*n_cal*(2.2-window_size)*fps)
spd_accuracy_code_looa = np.zeros(12)
spd_tps_code_looa = np.zeros(12)
spd_accuracy_looa = np.zeros(12)
batchsize = 64 #128 # 64 for burst
epoch = 20 #45 # 20 for burst

for i in range(12):
    print("TL to the participant : ", i)
    ind2take = [j for j in range(12) if j!=i]
    X = X_parent.copy()
    Y = Y_parent.copy()
    domains = domains_parent.copy()

    X_train = np.concatenate([np.concatenate(X[ind2take]).reshape(-1,X.shape[-2],X.shape[-1]),X[i][:nb_samples_cal]]).reshape(-1,X.shape[-2],X.shape[-1])
    Y_train = np.concatenate([np.concatenate(Y[ind2take]).reshape(-1),Y[i][:nb_samples_cal]]).reshape(-1)
    X_test = X[i][nb_samples_cal:]
    Y_test = Y[i][nb_samples_cal:]

    labels_code_test = labels_codes[i][n_cal*n_class:]
    print(labels_code_test.shape)

    X_std = X_train.std(axis=0)
    X_train /= X_std + 1e-8
    X_std = X_test.std(axis=0)
    X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    rus = RandomUnderSampler()
    counter=np.array(range(0,len(Y_train))).reshape(-1,1)
    index,_ = rus.fit_resample(counter,Y_train[:])
    X_train = np.squeeze(X_train[index,:,:], axis=1)
    Y_train = np.squeeze(Y_train[index])
    # print(Y_train)

    print(X_train[Y_train==1].shape)
    print(X_test.shape)

    print("Creating the different pipelines")
    lr = 1e-3
    # optimizer = riemannian_adam.RiemannianAdam(learning_rate=lr)
    # clf = SPDNet_AJD(n_epochs=epoch,batch_size=batchsize,valid_split=0.1)
    clf = SPDNet_Tensorflow(bimap_dims=[28,14,7])

    print("Fitting")
    weight_decay = 1e-4
    
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42, shuffle=True)
    history = clf.fit(np.array(x_train), y_train,
                    batch_size=batchsize, epochs=epoch,
                    validation_data=(np.array(x_val), y_val), shuffle=True)
    
    print("getting accuracy of participant ", i)
    y_pred = clf.predict(X_test)
    y_pred = np.array(y_pred)
    y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
    y_test_norm = np.array([0 if y == 0 else 1 for y in Y_test])

    tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
    spd_accuracy_looa[i] = balanced_accuracy_score(y_test_norm,y_pred_norm)
    print(spd_accuracy_looa[i])

    labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
        y_pred_norm, codes, min_len=30, sfreq=60, consecutive=50, window_size=window_size
    )
    spd_tps_code_looa[i] = np.mean(mean_long_accumul)
    spd_accuracy_code_looa[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    
    keras.backend.clear_session()
    del X_train
    del X_test
    del Y_train
    del Y_test

print(spd_accuracy_looa)
print(spd_accuracy_code_looa)
print(spd_tps_code_looa)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDNet/LOOA_score",spd_accuracy_looa)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDNet/LOOA_score_code",spd_accuracy_code_looa)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDNet/LOOA_tps_code",spd_tps_code_looa)


TL to the participant :  0
(56,)
balancing the number of ones and zeros
(13280, 32, 32)
(6552, 32, 32)
Creating the different pipelines
Fitting
Epoch 1/20
374/374 [==============================] - 6s 14ms/step - loss: 0.6393 - sparse_categorical_accuracy: 0.7077 - val_loss: 0.4526 - val_sparse_categorical_accuracy: 0.8053
Epoch 2/20
374/374 [==============================] - 6s 15ms/step - loss: 0.4161 - sparse_categorical_accuracy: 0.8169 - val_loss: 0.4299 - val_sparse_categorical_accuracy: 0.8020
Epoch 3/20
374/374 [==============================] - 5s 14ms/step - loss: 0.4003 - sparse_categorical_accuracy: 0.8235 - val_loss: 0.4302 - val_sparse_categorical_accuracy: 0.7978
Epoch 4/20
374/374 [==============================] - 5s 14ms/step - loss: 0.3958 - sparse_categorical_accuracy: 0.8233 - val_loss: 0.4138 - val_sparse_categorical_accuracy: 0.8136
Epoch 5/20
374/374 [==============================] - 5s 14ms/step - loss: 0.3946 - sparse_categorical_accuracy: 0.8251 - val_loss: 

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  7
(56,)
balancing the number of ones and zeros
(13280, 32, 32)
(6552, 32, 32)
Creating the different pipelines
Fitting
Epoch 1/20
374/374 [==============================] - 6s 14ms/step - loss: 0.6939 - sparse_categorical_accuracy: 0.7020 - val_loss: 0.4657 - val_sparse_categorical_accuracy: 0.7974
Epoch 2/20
374/374 [==============================] - 5s 14ms/step - loss: 0.4233 - sparse_categorical_accuracy: 0.8136 - val_loss: 0.4400 - val_sparse_categorical_accuracy: 0.7959
Epoch 3/20
374/374 [==============================] - 5s 14ms/step - loss: 0.4067 - sparse_categorical_accuracy: 0.8199 - val_loss: 0.4243 - val_sparse_categorical_accuracy: 0.8080
Epoch 4/20
374/374 [==============================] - 5s 14ms/step - loss: 0.4020 - sparse_categorical_accuracy: 0.8202 - val_loss: 0.4342 - val_sparse_categorical_accuracy: 0.8031
Epoch 5/20
374/374 [==============================] - 5s 14ms/step - loss: 0.3988 - sparse_categorical_accuracy: 0.8211 - val_loss: 

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  11
(56,)
balancing the number of ones and zeros
(13280, 32, 32)
(6552, 32, 32)
Creating the different pipelines
Fitting
Epoch 1/20
374/374 [==============================] - 7s 16ms/step - loss: 0.5464 - sparse_categorical_accuracy: 0.7526 - val_loss: 0.4156 - val_sparse_categorical_accuracy: 0.8227
Epoch 2/20
374/374 [==============================] - 6s 15ms/step - loss: 0.3956 - sparse_categorical_accuracy: 0.8291 - val_loss: 0.3887 - val_sparse_categorical_accuracy: 0.8215
Epoch 3/20
374/374 [==============================] - 6s 15ms/step - loss: 0.3828 - sparse_categorical_accuracy: 0.8325 - val_loss: 0.3765 - val_sparse_categorical_accuracy: 0.8362
Epoch 4/20
374/374 [==============================] - 6s 15ms/step - loss: 0.3783 - sparse_categorical_accuracy: 0.8345 - val_loss: 0.4067 - val_sparse_categorical_accuracy: 0.8110
Epoch 5/20
374/374 [==============================] - 6s 15ms/step - loss: 0.3775 - sparse_categorical_accuracy: 0.8344 - val_loss:

### Train Test

In [17]:
n_cal = 7
n_class = 4
spd_accuracy_code_perso = np.zeros(12)
spd_tps_code_perso = np.zeros(12)
spd_accuracy_perso = np.zeros(12)

for i in range(12):
    print("TL to the participant : ", i)
    X = X_parent.copy()
    Y = Y_parent.copy()
    domains = domains_parent.copy()
    nb_sample_cal = int(n_class*n_cal*(2.2-window_size)*fps)

    X_train = X[i][:nb_sample_cal]
    Y_train = Y[i][:nb_sample_cal]
    X_test = X[i][nb_sample_cal:]
    Y_test = Y[i][nb_sample_cal:]
    labels_code_test = labels_codes[i][(n_class*n_cal):]

    print(X_train.shape)
    print(X_test.shape)
    X_std = X_train.std(axis=0)
    X_train /= X_std + 1e-8
    X_std = X_test.std(axis=0)
    X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    rus = RandomUnderSampler()
    counter=np.array(range(0,len(Y_train))).reshape(-1,1)
    index,_ = rus.fit_resample(counter,Y_train[:])
    X_train = np.squeeze(X_train[index,:,:], axis=1)
    Y_train = np.squeeze(Y_train[index])

    print("Creating the different pipelines")
    lr = 1e-3
    # optimizer = riemannian_adam.RiemannianAdam(learning_rate=lr)
    batchsize = 64 #128 # 64 for burst
    epoch = 20 #45 # 20 for burst
    # clf = SPDNet_AJD(n_epochs=epoch,batch_size=batchsize,valid_split=0.1)
    clf = SPDNet_Tensorflow(bimap_dims=[28,14,7])

    print("Fitting")
    weight_decay = 1e-4
    
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42, shuffle=True)
    history = clf.fit(np.array(x_train), y_train,
                    batch_size=batchsize, epochs=epoch,
                    validation_data=(np.array(x_val), y_val), shuffle=True)
    # history = clf.fit(X_train,Y_train)

    # clf.fit_and_predict(X_train,Y_train,X_test,Y_test,"C:\\Users\\s.velut\\Documents\\These\\Protheus_PHD\\pictures\\SPD_Net")
    
    print("getting accuracy of participant ", i)
    y_pred = clf.predict(X_test)[:]
    y_pred = np.array(y_pred)
    y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
    y_test_norm = np.array([0 if y == 0 else 1 for y in Y_test])

    tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
    spd_accuracy_perso[i] = balanced_accuracy_score(y_test_norm,y_pred_norm)

    labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
        y_pred_norm, codes, min_len=30, sfreq=fps, consecutive=50, window_size=window_size
    )
    spd_tps_code_perso[i] = np.mean(mean_long_accumul)
    spd_accuracy_code_perso[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(spd_accuracy_perso)
print(spd_tps_code_perso)
print(spd_accuracy_code_perso)
# np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDNet/WO_score_cal1",spd_accuracy_perso)
# np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDNet/WO_score_code_cal1",spd_accuracy_code_perso)
# np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDNet/WO_tps_code_cal1",spd_tps_code_perso)

TL to the participant :  0
(3276, 32, 32)
(3744, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1/20
16/16 [==============================] - 1s 21ms/step - loss: 0.6489 - sparse_categorical_accuracy: 0.6528 - val_loss: 0.6241 - val_sparse_categorical_accuracy: 0.7054
Epoch 2/20
16/16 [==============================] - 0s 13ms/step - loss: 0.5922 - sparse_categorical_accuracy: 0.7470 - val_loss: 0.5943 - val_sparse_categorical_accuracy: 0.6875
Epoch 3/20
16/16 [==============================] - 0s 14ms/step - loss: 0.5489 - sparse_categorical_accuracy: 0.7976 - val_loss: 0.5650 - val_sparse_categorical_accuracy: 0.7411
Epoch 4/20
16/16 [==============================] - 0s 14ms/step - loss: 0.5181 - sparse_categorical_accuracy: 0.8244 - val_loss: 0.5248 - val_sparse_categorical_accuracy: 0.7857
Epoch 5/20
16/16 [==============================] - 0s 14ms/step - loss: 0.4870 - sparse_categorical_accuracy: 0.8304 - val_loss: 0.5061 - val_spar

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  9
(3276, 32, 32)
(3744, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1/20
16/16 [==============================] - 1s 28ms/step - loss: 2.4694 - sparse_categorical_accuracy: 0.4931 - val_loss: 1.0950 - val_sparse_categorical_accuracy: 0.5625
Epoch 2/20
16/16 [==============================] - 0s 17ms/step - loss: 0.8533 - sparse_categorical_accuracy: 0.5000 - val_loss: 0.6932 - val_sparse_categorical_accuracy: 0.4643
Epoch 3/20
16/16 [==============================] - 0s 16ms/step - loss: 0.6341 - sparse_categorical_accuracy: 0.6290 - val_loss: 0.6229 - val_sparse_categorical_accuracy: 0.6429
Epoch 4/20
16/16 [==============================] - 0s 16ms/step - loss: 0.5538 - sparse_categorical_accuracy: 0.7718 - val_loss: 0.5285 - val_sparse_categorical_accuracy: 0.7946
Epoch 5/20
16/16 [==============================] - 0s 17ms/step - loss: 0.4897 - sparse_categorical_accuracy: 0.8363 - val_loss: 0.4742 - val_spar

### LOO SPDBN net

In [14]:
n_cal = 7
n_class = 4
spdbn_accuracy_code_loo = np.zeros(12)
spdbn_tps_code_loo = np.zeros(12)
spdbn_accuracy_loo = np.zeros(12)

for i in range(12):
    print("TL to the participant : ", i)
    ind2take = [j for j in range(12) if j!=i]
    X = X_parent.copy()
    Y = Y_parent.copy()
    domains = domains_parent.copy()

    X_train = np.concatenate(X[ind2take]).reshape(-1,X.shape[-2],X.shape[-1])
    Y_train = np.concatenate(Y[ind2take]).reshape(-1)

    X_test = X[i]
    Y_test = Y[i]
    labels_code_test = labels_codes[i]

    print(X_train.shape)
    print(X_test.shape)
    X_std = X_train.std(axis=0)
    X_train /= X_std + 1e-8
    X_std = X_test.std(axis=0)
    X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    X_train, Y_train, domains_train = balance(X_train,Y_train,np.concatenate(domains[ind2take]))

    print("Creating the different pipelines")
    lr = 1e-3
    # optimizer = riemannian_adam.RiemannianAdam(learning_rate=lr)
    batchsize = 64 #128 # 64 for burst
    epoch = 20 #45 # 20 for burst
    # clf = SPDNet_AJD(n_epochs=epoch,batch_size=batchsize,valid_split=0.1)
    clf = SPDNetBN_Module(bimap_dims=[32,28,14,7])

    print("Fitting")
    weight_decay = 1e-4
    
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42, shuffle=True)

    # Convert data into PyTorch tensors
    X_train_tensor = torch.tensor(x_train, dtype=torch.float64)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    X_val_tensor = torch.tensor(x_val, dtype=torch.float64)
    y_val_tensor = torch.tensor(y_val, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float64)
    y_test_tensor = torch.tensor(Y_test, dtype=torch.long)

    # Create DataLoader for train, validation, and test sets
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    # Define loss function and optimizer
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch_riemannian_adam.RiemannianAdam(clf.parameters(), lr=0.001)

    # Train the model
    num_epochs = 20
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_dataloader:
            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = clf(inputs)
            loss = criterion(outputs.float(), labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_dataloader)}")

    print("Training finished!")

    # Validation
    clf.eval()
    val_correct = 0
    with torch.no_grad():
        for inputs, labels in val_dataloader:
            outputs = clf(inputs)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = val_correct / len(x_val)
    print(f"Validation Accuracy: {val_accuracy}")

    # Testing
    test_correct = 0
    y_pred= []
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            outputs = clf(inputs)
            _, predicted = torch.max(outputs, 1)
            y_pred.append(predicted)
            test_correct += (predicted == labels).sum().item()
            
    test_accuracy = test_correct / len(X_test)
    print(f"Test Accuracy: {test_accuracy}")

    # history = clf.fit(np.array(x_train), y_train,
    #                 batch_size=batchsize, epochs=epoch,
    #                 validation_data=(np.array(x_val), y_val), shuffle=True)
    # history = clf.fit(X_train,Y_train)

    # clf.fit_and_predict(X_train,Y_train,X_test,Y_test,"C:\\Users\\s.velut\\Documents\\These\\Protheus_PHD\\pictures\\SPD_Net")
    
    print("getting accuracy of participant ", i)
    y_pred = np.concatenate(y_pred)
    y_pred = np.array(y_pred)
    y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
    y_test_norm = np.array([0 if y == 0 else 1 for y in Y_test])

    tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
    spdbn_accuracy_loo[i] = balanced_accuracy_score(y_test_norm,y_pred_norm)

    labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
        y_pred_norm, codes, min_len=30, sfreq=fps, consecutive=50, window_size=window_size
    )
    spdbn_tps_code_loo[i] = np.mean(mean_long_accumul)
    spdbn_accuracy_code_loo[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(spdbn_accuracy_loo)
print(spdbn_tps_code_loo)
print(spdbn_accuracy_code_loo)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDbnNet/score/LOO_score",spdbn_accuracy_loo)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDbnNet/score_code/LOO_score_code",spdbn_accuracy_code_loo)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDbnNet/tps_code/LOO_tps_code",spdbn_tps_code_loo)

TL to the participant :  0
(77220, 32, 32)
(7020, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.5634700358715109
Epoch 2, Loss: 0.44464476518733526
Epoch 3, Loss: 0.4295818904234517
Epoch 4, Loss: 0.423150833335615
Epoch 5, Loss: 0.41887467466695333
Epoch 6, Loss: 0.4189846061811011
Epoch 7, Loss: 0.4108455642977709
Epoch 8, Loss: 0.413512312276389
Epoch 9, Loss: 0.4095538911079207
Epoch 10, Loss: 0.4108678188615589
Epoch 11, Loss: 0.41148876126414985
Epoch 12, Loss: 0.41089639644468984
Epoch 13, Loss: 0.40356355749310985
Epoch 14, Loss: 0.4048454445055736
Epoch 15, Loss: 0.40616481118304754
Epoch 16, Loss: 0.40351448468463397
Epoch 17, Loss: 0.3993104578105993
Epoch 18, Loss: 0.4039261411194519
Epoch 19, Loss: 0.40317589445139773
Epoch 20, Loss: 0.4041842158923867
Training finished!
Validation Accuracy: 0.7522727272727273
Test Accuracy: 0.8173789173789173
getting accuracy of participant  0
TL to the participant :  1
(77220, 32

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  2
(77220, 32, 32)
(7020, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.5033854902271302
Epoch 2, Loss: 0.42525800269457603
Epoch 3, Loss: 0.4137762847446626
Epoch 4, Loss: 0.408367454925532
Epoch 5, Loss: 0.40392307344303335
Epoch 6, Loss: 0.40284262729748604
Epoch 7, Loss: 0.4074221613506476
Epoch 8, Loss: 0.40115597275316073
Epoch 9, Loss: 0.39883300774199987
Epoch 10, Loss: 0.4009526347921741
Epoch 11, Loss: 0.3968185588957802
Epoch 12, Loss: 0.3972250913740486
Epoch 13, Loss: 0.39463438650452964
Epoch 14, Loss: 0.39497476828194433
Epoch 15, Loss: 0.40153950352662354
Epoch 16, Loss: 0.39410623775855186
Epoch 17, Loss: 0.3967924876318824
Epoch 18, Loss: 0.3923265592705819
Epoch 19, Loss: 0.39597263275295175
Epoch 20, Loss: 0.39577039743783654
Training finished!
Validation Accuracy: 0.8272727272727273
Test Accuracy: 0.8723646723646724
getting accuracy of participant  2
TL to the participant :  3
(77220

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  5
(77220, 32, 32)
(7020, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.5550778792590223
Epoch 2, Loss: 0.437459424538638
Epoch 3, Loss: 0.41709896593645057
Epoch 4, Loss: 0.4111791377266248
Epoch 5, Loss: 0.4061017848391046
Epoch 6, Loss: 0.40517345855953874
Epoch 7, Loss: 0.40333503432652
Epoch 8, Loss: 0.4001096150968024
Epoch 9, Loss: 0.40028271299376283
Epoch 10, Loss: 0.39917069789703175
Epoch 11, Loss: 0.3966162986252257
Epoch 12, Loss: 0.39606792267452007
Epoch 13, Loss: 0.3975441457042771
Epoch 14, Loss: 0.4015979729993369
Epoch 15, Loss: 0.4015612010994265
Epoch 16, Loss: 0.40117470142982337
Epoch 17, Loss: 0.39702465136845905
Epoch 18, Loss: 0.39925158404374633
Epoch 19, Loss: 0.3978756995451066
Epoch 20, Loss: 0.399929090973831
Training finished!
Validation Accuracy: 0.8166666666666667
Test Accuracy: 0.9091168091168091
getting accuracy of participant  5


...

TL to the participant :  9
(77220, 32, 32)
(7020, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.5233958943556714
Epoch 2, Loss: 0.4472290090815995
Epoch 3, Loss: 0.43656048915719475
Epoch 4, Loss: 0.4326792602897972
Epoch 5, Loss: 0.43001097200378297
Epoch 6, Loss: 0.426742056444768
Epoch 7, Loss: 0.4264805618152824
Epoch 8, Loss: 0.42367783665496817
Epoch 9, Loss: 0.4210352970147005
Epoch 10, Loss: 0.4220333884880748
Epoch 11, Loss: 0.4222307181566633
Epoch 12, Loss: 0.4196150243202204
Epoch 13, Loss: 0.4234148121328764
Epoch 14, Loss: 0.42048273680190884
Epoch 15, Loss: 0.4191270123326009
Epoch 16, Loss: 0.4209755794495665
Epoch 17, Loss: 0.4191803199549516
Epoch 18, Loss: 0.4162689439792146
Epoch 19, Loss: 0.41748848966052454
Epoch 20, Loss: 0.41984052303177055
Training finished!
Validation Accuracy: 0.8045454545454546
Test Accuracy: 0.8287749287749288
getting accuracy of participant  9
TL to the participant :  10
(77220, 32

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  11
(77220, 32, 32)
(7020, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.5423442182201211
Epoch 2, Loss: 0.44740292093446177
Epoch 3, Loss: 0.43372326208058226
Epoch 4, Loss: 0.4288491170893433
Epoch 5, Loss: 0.4263139457632137
Epoch 6, Loss: 0.42539377013842267
Epoch 7, Loss: 0.42257467733435733
Epoch 8, Loss: 0.42351756877796626
Epoch 9, Loss: 0.41875631562484206
Epoch 10, Loss: 0.414921468143822
Epoch 11, Loss: 0.4141502460484864
Epoch 12, Loss: 0.41568133835830995
Epoch 13, Loss: 0.4126357764646571
Epoch 14, Loss: 0.41470087475834355
Epoch 15, Loss: 0.41528721926833995
Epoch 16, Loss: 0.41155615977702603
Epoch 17, Loss: 0.4126286817494259
Epoch 18, Loss: 0.4123410131902464
Epoch 19, Loss: 0.4153753244267997
Epoch 20, Loss: 0.40984215058626666
Training finished!
Validation Accuracy: 0.6178030303030303
Test Accuracy: 0.8319088319088319
getting accuracy of participant  11


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


[0.82073454 0.74618127 0.82776632 0.80428265 0.77993127 0.86183849
 0.59553265 0.78371134 0.67746993 0.89375859 0.73061426 0.51627148]
[1.44       1.46333333 1.41855346 1.50681818 1.45578231 1.4172619
 1.3606383  1.44839744 1.43952381 1.45673077 1.47740741 1.3724359 ]
[0.83 0.83 0.98 0.9  0.93 0.93 0.68 0.9  0.75 1.   0.77 0.23]


### LOOA SPDBN net

In [15]:
n_cal = 7
n_class = 4
nb_samples_cal = int(n_class*n_cal*(2.2-window_size)*fps)
spdbn_accuracy_code_looa = np.zeros(12)
spdbn_tps_code_looa = np.zeros(12)
spdbn_accuracy_looa = np.zeros(12)

for i in range(12):
    print("TL to the participant : ", i)
    ind2take = [j for j in range(12) if j!=i]
    X = X_parent.copy()
    Y = Y_parent.copy()
    domains = domains_parent.copy()

    X_train = np.concatenate([np.concatenate(X[ind2take]).reshape(-1,X.shape[-2],X.shape[-1]),X[i][:nb_samples_cal]]).reshape(-1,X.shape[-2],X.shape[-1])
    Y_train = np.concatenate([np.concatenate(Y[ind2take]).reshape(-1),Y[i][:nb_samples_cal]]).reshape(-1)
    X_test = X[i][nb_samples_cal:]
    Y_test = Y[i][nb_samples_cal:]

    labels_code_test = labels_codes[i][n_cal*n_class:]

    print(X_train.shape)
    print(X_test.shape)
    X_std = X_train.std(axis=0)
    X_train /= X_std + 1e-8
    X_std = X_test.std(axis=0)
    X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    X_train, Y_train, domains_train = balance(X_train,Y_train,np.concatenate(domains[ind2take]))

    print("Creating the different pipelines")
    lr = 1e-3
    # optimizer = riemannian_adam.RiemannianAdam(learning_rate=lr)
    batchsize = 64 #128 # 64 for burst
    epoch = 20 #45 # 20 for burst
    # clf = SPDNet_AJD(n_epochs=epoch,batch_size=batchsize,valid_split=0.1)
    clf = SPDNetBN_Module(bimap_dims=[32,28,14,7])

    print("Fitting")
    weight_decay = 1e-4
    
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42, shuffle=True)

    # Convert data into PyTorch tensors
    X_train_tensor = torch.tensor(x_train, dtype=torch.float64)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    X_val_tensor = torch.tensor(x_val, dtype=torch.float64)
    y_val_tensor = torch.tensor(y_val, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float64)
    y_test_tensor = torch.tensor(Y_test, dtype=torch.long)

    # Create DataLoader for train, validation, and test sets
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    # Define loss function and optimizer
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch_riemannian_adam.RiemannianAdam(clf.parameters(), lr=0.001)

    # Train the model
    num_epochs = 20
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_dataloader:
            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = clf(inputs)
            loss = criterion(outputs.float(), labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_dataloader)}")

    print("Training finished!")

    # Validation
    clf.eval()
    val_correct = 0
    with torch.no_grad():
        for inputs, labels in val_dataloader:
            outputs = clf(inputs)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = val_correct / len(x_val)
    print(f"Validation Accuracy: {val_accuracy}")

    # Testing
    test_correct = 0
    y_pred= []
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            outputs = clf(inputs)
            _, predicted = torch.max(outputs, 1)
            y_pred.append(predicted)
            test_correct += (predicted == labels).sum().item()
            
    test_accuracy = test_correct / len(X_test)
    print(f"Test Accuracy: {test_accuracy}")

    # history = clf.fit(np.array(x_train), y_train,
    #                 batch_size=batchsize, epochs=epoch,
    #                 validation_data=(np.array(x_val), y_val), shuffle=True)
    # history = clf.fit(X_train,Y_train)

    # clf.fit_and_predict(X_train,Y_train,X_test,Y_test,"C:\\Users\\s.velut\\Documents\\These\\Protheus_PHD\\pictures\\SPDbn_Net")
    
    print("getting accuracy of participant ", i)
    y_pred = np.concatenate(y_pred)
    y_pred = np.array(y_pred)
    y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
    y_test_norm = np.array([0 if y == 0 else 1 for y in Y_test])

    tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
    spdbn_accuracy_looa[i] = balanced_accuracy_score(y_test_norm,y_pred_norm)

    labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
        y_pred_norm, codes, min_len=30, sfreq=fps, consecutive=50, window_size=window_size
    )
    spdbn_tps_code_looa[i] = np.mean(mean_long_accumul)
    spdbn_accuracy_code_looa[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(spdbn_accuracy_looa)
print(spdbn_tps_code_looa)
print(spdbn_accuracy_code_looa)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDbnNet/score/LOOA_score",spdbn_accuracy_looa)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDbnNet/score_code/LOOA_score_code",spdbn_accuracy_code_looa)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDbnNet/tps_code/LOOA_tps_code",spdbn_tps_code_looa)

TL to the participant :  0
(80496, 32, 32)
(3744, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.5050495002538927
Epoch 2, Loss: 0.42421202770163935
Epoch 3, Loss: 0.416910758782779
Epoch 4, Loss: 0.40735043064560944
Epoch 5, Loss: 0.40539501943895895
Epoch 6, Loss: 0.4027983453325046
Epoch 7, Loss: 0.40238970646294214
Epoch 8, Loss: 0.40172493742198073
Epoch 9, Loss: 0.3985906047045544
Epoch 10, Loss: 0.3965776303923258
Epoch 11, Loss: 0.3949703783277542
Epoch 12, Loss: 0.39454007393089674
Epoch 13, Loss: 0.3937151950373444
Epoch 14, Loss: 0.3893760667452889
Epoch 15, Loss: 0.39497111821847575
Epoch 16, Loss: 0.3947187374115631
Epoch 17, Loss: 0.39358547694420304
Epoch 18, Loss: 0.3913649208843708
Epoch 19, Loss: 0.39442520228124434
Epoch 20, Loss: 0.3938771156213617
Training finished!
Validation Accuracy: 0.8265151515151515
Test Accuracy: 0.8261217948717948
getting accuracy of participant  0
TL to the participant :  1
(80496, 

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  3
(80496, 32, 32)
(3744, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.5332964802941969
Epoch 2, Loss: 0.4457768581567272
Epoch 3, Loss: 0.4346119651711115
Epoch 4, Loss: 0.4320195717837221
Epoch 5, Loss: 0.4217670981281547
Epoch 6, Loss: 0.41148627994041287
Epoch 7, Loss: 0.40924247230092686
Epoch 8, Loss: 0.40542205030559214
Epoch 9, Loss: 0.4019500100484458
Epoch 10, Loss: 0.3982057410783024
Epoch 11, Loss: 0.39637031398152794
Epoch 12, Loss: 0.39922366199153725
Epoch 13, Loss: 0.3950126204759844
Epoch 14, Loss: 0.39561856361807035
Epoch 15, Loss: 0.39423030238318185
Epoch 16, Loss: 0.39493444445793346
Epoch 17, Loss: 0.39496070483038503
Epoch 18, Loss: 0.3883198008982725
Epoch 19, Loss: 0.39365143036489847
Epoch 20, Loss: 0.39012245653617766
Training finished!
Validation Accuracy: 0.5878787878787879
Test Accuracy: 0.2422542735042735
getting accuracy of participant  3


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  4
(80496, 32, 32)
(3744, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.540551817144758
Epoch 2, Loss: 0.4565057035094948
Epoch 3, Loss: 0.44430329666663243
Epoch 4, Loss: 0.43965422057656833
Epoch 5, Loss: 0.4301470824947921
Epoch 6, Loss: 0.42621738215287525
Epoch 7, Loss: 0.429647928844857
Epoch 8, Loss: 0.42374003077706984
Epoch 9, Loss: 0.42288678784364014
Epoch 10, Loss: 0.4224013418600123
Epoch 11, Loss: 0.4218965665066755
Epoch 12, Loss: 0.42129612221352514
Epoch 13, Loss: 0.4249708384515778
Epoch 14, Loss: 0.4187292326842585
Epoch 15, Loss: 0.42388382513997375
Epoch 16, Loss: 0.4272517065607732
Epoch 17, Loss: 0.4221236377313573
Epoch 18, Loss: 0.4189682952578991
Epoch 19, Loss: 0.42368222925291266
Epoch 20, Loss: 0.42017638939683155
Training finished!
Validation Accuracy: 0.7708333333333334
Test Accuracy: 0.7831196581196581
getting accuracy of participant  4
TL to the participant :  5
(80496, 3

### Train Test SPDBN net

In [ ]:
n_cal = 7
n_class = 4
spdbn_accuracy_code_perso = np.zeros(12)
spdbn_tps_code_perso = np.zeros(12)
spdbn_accuracy_perso = np.zeros(12)

for i in range(12):
    print("TL to the participant : ", i)
    X = X_parent.copy()
    Y = Y_parent.copy()
    domains = domains_parent.copy()
    nb_sample_cal = int(n_class*n_cal*(2.2-window_size)*fps)

    X_train = X[i][:nb_sample_cal]
    Y_train = Y[i][:nb_sample_cal]
    X_test = X[i][nb_sample_cal:]
    Y_test = Y[i][nb_sample_cal:]
    labels_code_test = labels_codes[i][(n_class*n_cal):]

    print(X_train.shape)
    print(X_test.shape)
    X_std = X_train.std(axis=0)
    X_train /= X_std + 1e-8
    X_std = X_test.std(axis=0)
    X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    X_train, Y_train, domains_train = balance(X_train,Y_train,domains[i][:nb_sample_cal])

    print("Creating the different pipelines")
    lr = 1e-3
    # optimizer = riemannian_adam.RiemannianAdam(learning_rate=lr)
    batchsize = 64 #128 # 64 for burst
    epoch = 20 #45 # 20 for burst
    # clf = SPDNet_AJD(n_epochs=epoch,batch_size=batchsize,valid_split=0.1)
    clf = SPDNetBN_Module(bimap_dims=[32,28,14,7])

    print("Fitting")
    weight_decay = 1e-4
    
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42, shuffle=True)

    # Convert data into PyTorch tensors
    X_train_tensor = torch.tensor(x_train, dtype=torch.float64)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    X_val_tensor = torch.tensor(x_val, dtype=torch.float64)
    y_val_tensor = torch.tensor(y_val, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float64)
    y_test_tensor = torch.tensor(Y_test, dtype=torch.long)

    # Create DataLoader for train, validation, and test sets
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
    val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    # Define loss function and optimizer
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch_riemannian_adam.RiemannianAdam(clf.parameters(), lr=0.001)

    # Train the model
    num_epochs = 20
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_dataloader:
            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            # print(inputs.shape)
            # print(labels.shape)
            outputs = clf(inputs)
            loss = criterion(outputs.float(), labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_dataloader)}")

    print("Training finished!")

    # Validation
    clf.eval()
    val_correct = 0
    with torch.no_grad():
        for inputs, labels in val_dataloader:
            outputs = clf(inputs)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = val_correct / len(x_val)
    print(f"Validation Accuracy: {val_accuracy}")

    # Testing
    test_correct = 0
    y_pred= []
    with torch.no_grad():
        for inputs, labels in test_dataloader:
            outputs = clf(inputs)
            _, predicted = torch.max(outputs, 1)
            y_pred.append(predicted)
            test_correct += (predicted == labels).sum().item()
            
    test_accuracy = test_correct / len(X_test)
    print(f"Test Accuracy: {test_accuracy}")

    # history = clf.fit(np.array(x_train), y_train,
    #                 batch_size=batchsize, epochs=epoch,
    #                 validation_data=(np.array(x_val), y_val), shuffle=True)
    # history = clf.fit(X_train,Y_train)

    # clf.fit_and_predict(X_train,Y_train,X_test,Y_test,"C:\\Users\\s.velut\\Documents\\These\\Protheus_PHD\\pictures\\SPDbn_Net")
    
    print("getting accuracy of participant ", i)
    y_pred = np.concatenate(y_pred)
    y_pred = np.array(y_pred)
    y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
    y_test_norm = np.array([0 if y == 0 else 1 for y in Y_test])

    tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
    spdbn_accuracy_perso[i] = balanced_accuracy_score(y_test_norm,y_pred_norm)

    labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
        y_pred_norm, codes, min_len=30, sfreq=fps, consecutive=50, window_size=window_size
    )
    spdbn_tps_code_perso[i] = np.mean(mean_long_accumul)
    spdbn_accuracy_code_perso[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(spdbn_accuracy_perso)
print(spdbn_tps_code_perso)
print(spdbn_accuracy_code_perso)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDbnNet/score/WO_score",spdbn_accuracy_perso)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDbnNet/score_code/WO_score_code",spdbn_accuracy_code_perso)
np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDbnNet/tps_code/WO_tps_code",spdbn_tps_code_perso)

TL to the participant :  0
(3276, 32, 32)
(3744, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.6823236718773842
Epoch 2, Loss: 0.649629008024931
Epoch 3, Loss: 0.6083479635417461
Epoch 4, Loss: 0.5643698871135712
Epoch 5, Loss: 0.5281348172575235
Epoch 6, Loss: 0.4974643811583519
Epoch 7, Loss: 0.45665474236011505
Epoch 8, Loss: 0.42483285069465637
Epoch 9, Loss: 0.40259677171707153
Epoch 10, Loss: 0.39244604110717773
Epoch 11, Loss: 0.38086641393601894
Epoch 12, Loss: 0.3690508082509041
Epoch 13, Loss: 0.358419444411993
Epoch 14, Loss: 0.34972649440169334
Epoch 15, Loss: 0.34928514808416367
Epoch 16, Loss: 0.33787946216762066
Epoch 17, Loss: 0.3362749759107828
Epoch 18, Loss: 0.33344438299536705
Epoch 19, Loss: 0.3345995619893074
Epoch 20, Loss: 0.3245616042986512
Training finished!
Validation Accuracy: 0.8660714285714286
Test Accuracy: 0.8042200854700855
getting accuracy of participant  0
TL to the participant :  1
(3276, 32,

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  11
(3276, 32, 32)
(3744, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.6892720051109791
Epoch 2, Loss: 0.6618945002555847
Epoch 3, Loss: 0.6419619359076023
Epoch 4, Loss: 0.6218759007751942
Epoch 5, Loss: 0.5998190455138683
Epoch 6, Loss: 0.581920437514782
Epoch 7, Loss: 0.5612229518592358
Epoch 8, Loss: 0.5437341388314962
Epoch 9, Loss: 0.5280359480530024
Epoch 10, Loss: 0.5196700170636177
Epoch 11, Loss: 0.511547140777111
Epoch 12, Loss: 0.5003367587924004
Epoch 13, Loss: 0.4887511469423771
Epoch 14, Loss: 0.48168879374861717
Epoch 15, Loss: 0.47739036940038204
Epoch 16, Loss: 0.4689680840820074
Epoch 17, Loss: 0.46069794334471226
Epoch 18, Loss: 0.467211052775383
Epoch 19, Loss: 0.45860209316015244
Epoch 20, Loss: 0.45586597360670567
Training finished!
Validation Accuracy: 0.7589285714285714
Test Accuracy: 0.5798611111111112
getting accuracy of participant  11
[0.82983247 0.75774001 0.8451192  0.7665

In [16]:
inputs.shape

torch.Size([64, 32, 32])

### RPA transfer learning

In [26]:
def get_scores(source_org,target_org_train,target_org_test):
    # create a scores dictionary
    methods_list = ['org', 'rct', 'rpa', 'clb']
    scores = OrderedDict()
    for method in methods_list:
        scores[method] = []

    nrzt = 10
    for _ in range(nrzt):

        # get the split for the source and target dataset
        # source_org, target_org_train, target_org_test = TL.get_sourcetarget_split(source, target, ncovs_train=10)

        # get the score with the original dataset
        print("get the score with the original dataset")
        scores['org'].append(TL.get_score_transferlearning(clf, source_org, target_org_train, target_org_test))

        # get the score with the re-centered matrices
        print("get the score with the re-centered matrices")
        source_rct, target_rct_train, target_rct_test = TL.RPA_recenter(source_org, target_org_train, target_org_test)
        scores['rct'].append(TL.get_score_transferlearning(clf, source_rct, target_rct_train, target_rct_test))

        # rotate the re-centered-stretched matrices using information from classes
        print("get the score with the rotated matrices")
        source_rpa, target_rpa_train, target_rpa_test = TL.RPA_rotate(source_rct, target_rct_train, target_rct_test,distance="rie")
        scores['rpa'].append(TL.get_score_transferlearning(clf, source_rpa, target_rpa_train, target_rpa_test))

        # get the score without any transformation
        print("get the score with the no_transfo matrices")
        scores['clb'].append(TL.get_score_notransfer(clf, target_org_train, target_org_test))
        
    for method in methods_list:
        scores[method] = np.mean(scores[method])

    return scores

In [27]:
def get_ypred(source_org,target_org_train,target_org_test,clf):
    # create a scores dictionary
    methods_list = ['org', 'rct', 'rpa', 'clb']
    y_pred = OrderedDict()
    for method in methods_list:
        y_pred[method] = []

    nrzt = 1
    for _ in range(nrzt):

        # get the split for the source and target dataset
        # source_org, target_org_train, target_org_test = TL.get_sourcetarget_split(source, target, ncovs_train=10)

        # get the y_pred with the original dataset
        print("get the y_pred with the original dataset")
        y_pred['org'].append(TL.get_y_pred_transferlearning(clone(clf), source_org, target_org_train, target_org_test))

        # get the y_pred with the re-centered matrices
        print("Recenter the martices")
        source_rct, target_rct_train, target_rct_test = TL.RPA_recenter(source_org, target_org_train, target_org_test)
        print("get the y_pred with the re-centered matrices")
        y_pred['rct'].append(TL.get_y_pred_transferlearning(clone(clf), source_rct, target_rct_train, target_rct_test))

        # rotate the re-centered-stretched matrices using information from classes
        print("Rotating the matices")
        source_rpa, target_rpa_train, target_rpa_test = TL.RPA_rotate(source_rct, target_rct_train, target_rct_test,distance="rie")
        print("get the y_pred with the rotated matrices")
        y_pred['rpa'].append(TL.get_y_pred_transferlearning(clone(clf), source_rpa, target_rpa_train, target_rpa_test))

        # get the y_pred without any transformation
        print("get the y_pred with the no_transfo matrices")
        y_pred['clb'].append(TL.get_y_pred_notransfer(clone(clf), target_org_train, target_org_test))
        
    # for method in methods_list:
    #     y_pred[method] = np.mean(y_pred[method])

    return y_pred

In [28]:
n_cal = 4
n_class = 4
methods_list = ['org', 'rct', 'rpa', 'clb']
nb_samples_cal = int(n_class*n_cal*(2.2-window_size)*fps)

spd_accuracy_code_rpa = OrderedDict()
spd_accuracy_rpa = OrderedDict()
spd_tps_code_rpa = OrderedDict()
for method in methods_list:
    spd_accuracy_code_rpa[method] = np.zeros(12)
    spd_accuracy_rpa[method] = np.zeros(12)
    spd_tps_code_rpa[method] = np.zeros(12)

for i in range(12):
    print("TL to the participant : ", i)
    # ind2take = [2,3]
    X = X_parent.copy()
    Y = Y_parent.copy()
    domains = domains_parent.copy()
    ind2take = [j for j in range(12) if j!=i]

    X_source_org = np.concatenate(X[ind2take]).reshape(-1,X.shape[-2],X.shape[-1])
    Y_source_org = np.concatenate(Y[ind2take]).reshape(-1)
    X_source_org,Y_source_org,domains_source = balance(X_source_org,Y_source_org, np.concatenate(domains[ind2take]))

    X_target_train_org, Y_target_train_org = X[i][:nb_samples_cal],Y[i][:nb_samples_cal]
    X_target_train_org,Y_target_train_org,domains_target_train = balance(X_target_train_org,Y_target_train_org, domains[i][:nb_samples_cal])

    X_target_test_org, Y_target_test_org = X[i][nb_samples_cal:], Y[i][nb_samples_cal:]
    # X_target_org = xdawncov.transform(X_target_org)
    labels_code_test = labels_codes[i][(n_class*n_cal):]

    print(X_source_org.shape)
    print(labels_code_test.shape)
    print(X_target_train_org.shape)
    print(X_target_test_org.shape)
    X_std = X_source_org.std(axis=0)
    X_source_org /= X_std + 1e-8
    X_std = X_target_train_org.std(axis=0)
    X_target_train_org /= X_std + 1e-8
    X_std = X_target_test_org.std(axis=0)
    X_target_test_org /= X_std + 1e-8

    print("Creating the dic")
    source_org = {}
    source_org['covs'] = X_source_org
    source_org['labels'] = Y_source_org
    target_org_train = {}
    target_org_train['covs'] = X_target_train_org
    target_org_train['labels'] = Y_target_train_org
    target_org_test = {}
    target_org_test['covs'] = X_target_test_org
    target_org_test['labels'] = Y_target_test_org

    print("Creating the different pipelines")
    lr = 1e-3
    # optimizer = riemannian_adam.RiemannianAdam(learning_rate=lr)
    batchsize = 64 #128 # 64 for burst
    epoch = 20 #45 # 20 for burst
    # clf = SPDNet_AJD(n_epochs=epoch,batch_size=batchsize,valid_split=0.1)
    clf = SPDNet_Tensorflow(bimap_dims=[28,14,7])

    print("Fitting")
    weight_decay = 1e-4

    # X_source_org, x_val, Y_source_org, y_val = train_test_split(X_source_org, Y_source_org, test_size=0.1, random_state=42, shuffle=True)
    # history = clf.fit(np.array(X_source_org), Y_source_org,
    #                 batch_size=batchsize, epochs=epoch,
    #                 validation_data=(np.array(x_val), y_val), shuffle=True)

    # # clf.fit_and_predict(X_source_org,Y_source_org,X_target_org,Y_test,"C:\\Users\\s.velut\\Documents\\These\\Protheus_PHD\\pictures\\SPD_Net")

    # print("getting accuracy of participant ", i)
    # y_pred = clf.predict(X_target_org)[:]
    # y_pred = np.array(y_pred)
    y_pred = get_ypred(source_org,target_org_train,target_org_test,clf)

    for method in methods_list:
        y_pred_temp = y_pred[method]
        # print(y_pred_temp)
        y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred_temp[0]])
        y_test_norm = np.array([0 if y == 0 else 1 for y in Y_target_test_org])

        tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
        spd_accuracy_rpa[method][i] = balanced_accuracy_score(y_test_norm,y_pred_norm)
        

        labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
            y_pred_norm, codes, min_len=30, sfreq=fps, consecutive=50, window_size=window_size
        )
        spd_tps_code_rpa[method][i] = np.mean(mean_long_accumul)
        spd_accuracy_code_rpa[method][i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(spd_accuracy_rpa)
print(spd_tps_code_rpa)
print(spd_accuracy_code_rpa)
for m in methods_list:
    np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDNet/RPA{}_score".format(m),spd_accuracy_rpa[m])
    np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDNet/RPA{}_score_code".format(m),spd_accuracy_code_rpa[m])
    np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDNet/RPA{}_tps_code".format(m),spd_tps_code_rpa[m])

TL to the participant :  0
(26400, 32, 32)
(44,)
(640, 32, 32)
(5148, 32, 32)
Creating the dic
Creating the different pipelines
Fitting
get the y_pred with the original dataset
shape of training X (27040, 32, 32)
shape of training Y (27040,)
Epoch 1/20
381/381 [==============================] - 8s 18ms/step - loss: 0.6106 - sparse_categorical_accuracy: 0.7382 - val_loss: 0.4304 - val_sparse_categorical_accuracy: 0.8147
Epoch 2/20
381/381 [==============================] - 7s 18ms/step - loss: 0.4101 - sparse_categorical_accuracy: 0.8175 - val_loss: 0.4192 - val_sparse_categorical_accuracy: 0.8073
Epoch 3/20
381/381 [==============================] - 7s 18ms/step - loss: 0.4005 - sparse_categorical_accuracy: 0.8215 - val_loss: 0.4082 - val_sparse_categorical_accuracy: 0.8129
Epoch 4/20
381/381 [==============================] - 7s 18ms/step - loss: 0.3961 - sparse_categorical_accuracy: 0.8230 - val_loss: 0.4045 - val_sparse_categorical_accuracy: 0.8180
Epoch 5/20
381/381 [==============

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  1
(26400, 32, 32)
(44,)
(640, 32, 32)
(5148, 32, 32)
Creating the dic
Creating the different pipelines
Fitting
get the y_pred with the original dataset
shape of training X (27040, 32, 32)
shape of training Y (27040,)
Epoch 1/20
381/381 [==============================] - 8s 19ms/step - loss: 0.5668 - sparse_categorical_accuracy: 0.7456 - val_loss: 0.4321 - val_sparse_categorical_accuracy: 0.8192
Epoch 2/20
381/381 [==============================] - 8s 20ms/step - loss: 0.4027 - sparse_categorical_accuracy: 0.8252 - val_loss: 0.4148 - val_sparse_categorical_accuracy: 0.8088
Epoch 3/20
381/381 [==============================] - 7s 19ms/step - loss: 0.3892 - sparse_categorical_accuracy: 0.8287 - val_loss: 0.4049 - val_sparse_categorical_accuracy: 0.8184
Epoch 4/20
381/381 [==============================] - 7s 19ms/step - loss: 0.3858 - sparse_categorical_accuracy: 0.8307 - val_loss: 0.4009 - val_sparse_categorical_accuracy: 0.8195
Epoch 5/20
381/381 [==============

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  2
(26400, 32, 32)
(44,)
(640, 32, 32)
(5148, 32, 32)
Creating the dic
Creating the different pipelines
Fitting
get the y_pred with the original dataset
shape of training X (27040, 32, 32)
shape of training Y (27040,)
Epoch 1/20
381/381 [==============================] - 8s 19ms/step - loss: 0.5633 - sparse_categorical_accuracy: 0.7317 - val_loss: 0.4685 - val_sparse_categorical_accuracy: 0.7763
Epoch 2/20
381/381 [==============================] - 7s 19ms/step - loss: 0.4053 - sparse_categorical_accuracy: 0.8247 - val_loss: 0.4667 - val_sparse_categorical_accuracy: 0.7722
Epoch 3/20
381/381 [==============================] - 7s 19ms/step - loss: 0.3892 - sparse_categorical_accuracy: 0.8265 - val_loss: 0.4150 - val_sparse_categorical_accuracy: 0.8110
Epoch 4/20
381/381 [==============================] - 7s 19ms/step - loss: 0.3855 - sparse_categorical_accuracy: 0.8309 - val_loss: 0.4147 - val_sparse_categorical_accuracy: 0.8088
Epoch 5/20
381/381 [==============

...

TL to the participant :  9
(26400, 32, 32)
(44,)
(640, 32, 32)
(5148, 32, 32)
Creating the dic
Creating the different pipelines
Fitting
get the y_pred with the original dataset
shape of training X (27040, 32, 32)
shape of training Y (27040,)
Epoch 1/20
381/381 [==============================] - 13s 32ms/step - loss: 0.6035 - sparse_categorical_accuracy: 0.6901 - val_loss: 0.4687 - val_sparse_categorical_accuracy: 0.8047
Epoch 2/20
381/381 [==============================] - 12s 31ms/step - loss: 0.4309 - sparse_categorical_accuracy: 0.8102 - val_loss: 0.4166 - val_sparse_categorical_accuracy: 0.8125
Epoch 3/20
381/381 [==============================] - 12s 31ms/step - loss: 0.4099 - sparse_categorical_accuracy: 0.8173 - val_loss: 0.4232 - val_sparse_categorical_accuracy: 0.8003
Epoch 4/20
381/381 [==============================] - 12s 32ms/step - loss: 0.4048 - sparse_categorical_accuracy: 0.8187 - val_loss: 0.4305 - val_sparse_categorical_accuracy: 0.7992
Epoch 5/20
381/381 [==========

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\

TL to the participant :  10
(26400, 32, 32)
(44,)
(640, 32, 32)
(5148, 32, 32)
Creating the dic
Creating the different pipelines
Fitting
get the y_pred with the original dataset
shape of training X (27040, 32, 32)
shape of training Y (27040,)
Epoch 1/20
381/381 [==============================] - 11s 28ms/step - loss: 1.0056 - sparse_categorical_accuracy: 0.6649 - val_loss: 0.5103 - val_sparse_categorical_accuracy: 0.7563
Epoch 2/20
381/381 [==============================] - 10s 27ms/step - loss: 0.4413 - sparse_categorical_accuracy: 0.8069 - val_loss: 0.4323 - val_sparse_categorical_accuracy: 0.8070
Epoch 3/20
381/381 [==============================] - 11s 28ms/step - loss: 0.4075 - sparse_categorical_accuracy: 0.8175 - val_loss: 0.4075 - val_sparse_categorical_accuracy: 0.8169
Epoch 4/20
381/381 [==============================] - 11s 28ms/step - loss: 0.3979 - sparse_categorical_accuracy: 0.8206 - val_loss: 0.4342 - val_sparse_categorical_accuracy: 0.8025
Epoch 5/20
381/381 [=========

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


OrderedDict([('org', array([0.83717198, 0.76288074, 0.82919986, 0.79581186, 0.83962043,
       0.87094658, 0.81957006, 0.87913543, 0.71942948, 0.93896439,
       0.79281865, 0.72427366])), ('rct', array([0.80106607, 0.76592666, 0.80250117, 0.72967432, 0.72301429,
       0.78218135, 0.71524133, 0.73549672, 0.74261364, 0.8394447 ,
       0.78652765, 0.72632381])), ('rpa', array([0.69844189, 0.60848758, 0.71359536, 0.66833411, 0.76070759,
       0.80476218, 0.72731959, 0.6294283 , 0.570847  , 0.82470712,
       0.65812441, 0.65475633])), ('clb', array([0.82686856, 0.74857076, 0.8168873 , 0.74874063, 0.77379335,
       0.88291354, 0.81122891, 0.87720244, 0.58301312, 0.90600398,
       0.7912664 , 0.58579545]))])
OrderedDict([('org', array([1.40540541, 1.48198198, 1.43063063, 1.4625    , 1.41574074,
       1.42342342, 1.4377451 , 1.385     , 1.39166667, 1.41937984,
       1.44333333, 1.44679487])), ('rct', array([1.43245614, 1.41236559, 1.41965812, 1.48952381, 1.41071429,
       1.43823529,

### RPA from pyriemann learning

In [19]:
def get_ypred_pyriemann(X_org,Y_org,Y_enc,domain,target_domain,clf):
    # create a scores dictionary
    methods_list = ['org', 'rct', 'rot']
    y_pred = OrderedDict()
    for method in methods_list:
        y_pred[method] = []

    # get the y_pred with the original dataset
    print("get the y_pred with the original dataset")
    y_pred['org'].append(get_y_pred(clf, X_org[domain!=target_domain], Y_org[domain!=target_domain], X_org[domain==target_domain], Y_org[domain==target_domain]))

    # get the y_pred with the re-centered matrices
    print("Recenter the martices")
    rct = TLCenter(target_domain=target_domain)
    X_rct = rct.fit_transform(X_org, Y_enc)
    Y_rct = Y_org
    print("get the y_pred with the re-centered matrices")
    y_pred['rct'].append(get_y_pred(clf, X_rct[domain!=target_domain], Y_rct[domain!=target_domain], X_rct[domain==target_domain], Y_rct[domain==target_domain]))
    # X_rct = X_org.copy()
    # Y_rct = Y_org.copy()

    # rotate the re-centered-stretched matrices using information from classes
    print("Rotating the matices")
    rot = TLRotate(target_domain=target_domain, metric='riemann')
    X_rot = rot.fit_transform(X_rct, Y_enc)
    Y_rot = Y_rct
    print("get the y_pred with the rotated matrices")
    y_pred['rot'].append(get_y_pred(clf, X_rot[domain!=target_domain], Y_rot[domain!=target_domain], X_rot[domain==target_domain], Y_rot[domain==target_domain]))
        
    # for method in methods_list:
    #     y_pred[method] = np.mean(y_pred[method])

    return y_pred

In [20]:
X_parent.shape

(12, 7020, 32, 32)

In [21]:
n_cal = 4
n_class = 4
methods_list = ['org', 'rct', 'rot']
nb_samples_cal = int(n_class*n_cal*(2.2-window_size)*fps)

spd_accuracy_code_rpa = OrderedDict()
spd_accuracy_rpa = OrderedDict()
spd_tps_code_rpa = OrderedDict()
for method in methods_list:
    spd_accuracy_code_rpa[method] = np.zeros(12)
    spd_accuracy_rpa[method] = np.zeros(12)
    spd_tps_code_rpa[method] = np.zeros(12)

for i in range(12):
    print("TL to the participant : ", i)
    # ind2take = [2,3]
    X = X_parent.copy()
    Y = Y_parent.copy()
    domains = domains_parent.copy().reshape(-1)
    Y_org = Y.reshape(-1)
    X_org = X.reshape(-1,X.shape[-2],X.shape[-1])
    labels_code_test = labels_codes[i]



    print(X_org.shape)
    print(domains.shape)
    X_std = X.std(axis=0)
    X /= X_std + 1e-8

    X_enc, Y_enc = encode_domains(X_org,Y_org,domains)

    clf = SPDNet_Tensorflow(bimap_dims=[28,14,7])

    y_pred = get_ypred_pyriemann(X_org,Y_org,Y_enc,domains,"Source_sub_{}".format(i),clf)


    for method in methods_list:
        y_pred_temp = y_pred[method]
        # print(y_pred_temp)
        y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred_temp[0]])
        y_test_norm = np.array([0 if y == 0 else 1 for y in Y_org[domains=="Source_sub_{}".format(i)]])

        tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
        spd_accuracy_rpa[method][i] = balanced_accuracy_score(y_test_norm,y_pred_norm)
        print(spd_accuracy_rpa[method][i])
        

        labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
            y_pred_norm, codes, min_len=30, sfreq=freq, consecutive=50, window_size=window_size
        )
        spd_tps_code_rpa[method][i] = np.mean(mean_long_accumul)
        spd_accuracy_code_rpa[method][i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)

        # np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDNet/RPA{}_score_{}".format(method,i),spd_accuracy_rpa[method])
        # np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDNet/RPA{}_score_code_{}".format(method,i),spd_accuracy_code_rpa[method])
        # np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDNet/RPA{}_tps_code_{}".format(method,i),spd_tps_code_rpa[method])
    keras.backend.clear_session()

print(spd_accuracy_rpa)
print(spd_tps_code_rpa)
print(spd_accuracy_code_rpa)
for m in methods_list:
    np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDNet/RPApyriem{}_score".format(m),spd_accuracy_rpa[m])
    np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDNet/RPApyriem{}_score_code".format(m),spd_accuracy_code_rpa[m])
    np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/results/Score_TF/SPDNet/RPApyriem{}_tps_code".format(m),spd_tps_code_rpa[m])

TL to the participant :  0
(84240, 32, 32)
(84240,)
get the y_pred with the original dataset
Epoch 1/20
1086/1086 [==============================] - 17s 15ms/step - loss: 0.3504 - sparse_categorical_accuracy: 0.8573 - val_loss: 0.3022 - val_sparse_categorical_accuracy: 0.8667
Epoch 2/20
1086/1086 [==============================] - 16s 15ms/step - loss: 0.2786 - sparse_categorical_accuracy: 0.8830 - val_loss: 0.2754 - val_sparse_categorical_accuracy: 0.8868
Epoch 3/20
1086/1086 [==============================] - 16s 14ms/step - loss: 0.2747 - sparse_categorical_accuracy: 0.8849 - val_loss: 0.2712 - val_sparse_categorical_accuracy: 0.8884
Epoch 4/20
1086/1086 [==============================] - 16s 15ms/step - loss: 0.2723 - sparse_categorical_accuracy: 0.8860 - val_loss: 0.2708 - val_sparse_categorical_accuracy: 0.8912
Epoch 5/20
1086/1086 [==============================] - 17s 16ms/step - loss: 0.2703 - sparse_categorical_accuracy: 0.8866 - val_loss: 0.2695 - val_sparse_categorical_accu

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyriemann\transfer\_rotate.py:132: UserWarning: Convergence not reached.
  warnings.warn('Convergence not reached.')
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyriemann\transfer\_estimators.py:586: ComplexWarning: Casting complex values to real discards the imaginary part
  X_rot[idx] = self.rotations_[d] @ X[idx] @ self.rotations_[d].T


get the y_pred with the rotated matrices
Epoch 1/20
1086/1086 [==============================] - 17s 15ms/step - loss: 0.3015 - sparse_categorical_accuracy: 0.8742 - val_loss: 0.2832 - val_sparse_categorical_accuracy: 0.8807
Epoch 2/20
1086/1086 [==============================] - 17s 15ms/step - loss: 0.2738 - sparse_categorical_accuracy: 0.8872 - val_loss: 0.2761 - val_sparse_categorical_accuracy: 0.8846
Epoch 3/20
1086/1086 [==============================] - 17s 15ms/step - loss: 0.2717 - sparse_categorical_accuracy: 0.8881 - val_loss: 0.2689 - val_sparse_categorical_accuracy: 0.8916
Epoch 4/20
1086/1086 [==============================] - 18s 16ms/step - loss: 0.2691 - sparse_categorical_accuracy: 0.8893 - val_loss: 0.2674 - val_sparse_categorical_accuracy: 0.8904
Epoch 5/20
1086/1086 [==============================] - 17s 15ms/step - loss: 0.2680 - sparse_categorical_accuracy: 0.8907 - val_loss: 0.2729 - val_sparse_categorical_accuracy: 0.8857
Epoch 6/20
1086/1086 [=================

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


0.7760609965635739


...

0.6362886597938144


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


0.6656056701030928


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


OrderedDict([('org', array([0.80758162, 0.6738445 , 0.7618256 , 0.61725086, 0.79695876,
       0.79492268, 0.76766323, 0.84178694, 0.57879725, 0.80977663,
       0.75089347, 0.64729381])), ('rct', array([0.776061  , 0.69810137, 0.73943729, 0.70049399, 0.73378866,
       0.79318299, 0.75450172, 0.76842784, 0.61690292, 0.86652921,
       0.7254854 , 0.63628866])), ('rot', array([0.72417096, 0.72652062, 0.7247079 , 0.71513746, 0.78853093,
       0.82364261, 0.79437285, 0.8492354 , 0.59807131, 0.88451031,
       0.7660567 , 0.66560567]))])
OrderedDict([('org', array([1.42352941, 1.3937037 , 1.41388889, 1.43484848, 1.40666667,
       1.41602564, 1.4       , 1.37647059, 1.38695652, 1.4139881 ,
       1.45902778, 1.43682171])), ('rct', array([1.39233333, 1.41666667, 1.4085034 , 1.47619048, 1.44      ,
       1.42222222, 1.44432624, 1.415625  , 1.42458333, 1.44502924,
       1.45606061, 1.46259259])), ('rot', array([1.42232704, 1.40902778, 1.43410853, 1.52222222, 1.39791667,
       1.4077381 ,

#### Test RPA

In [ ]:
n_cal = 2
n_class = 4
methods_list = ['org', 'rct', 'rpa', 'clb']
nb_samples_cal = int(n_class*n_cal*(2.2-window_size)*fps)

spd_accuracy_code_rpa = OrderedDict()
spd_accuracy_rpa = OrderedDict()
y_pred = OrderedDict()

spd_tps_code_rpa = OrderedDict()
for method in methods_list:
    spd_accuracy_code_rpa[method] = np.zeros(12)
    spd_accuracy_rpa[method] = np.zeros(12)
    y_pred[method] = []
    spd_tps_code_rpa[method] = np.zeros(12)

shape_target = 300

for i in range(3):
    print("TL to the participant : ", i)
    # ind2take = [2,3]
    X = X_parent.copy()
    Y = Y_parent.copy()
    domains = domains_parent.copy()
    ind2take = [i]
    # ind2take = [j for j in range(3)]

    X_source_org = np.concatenate(X[ind2take]).reshape(-1,X.shape[-2],X.shape[-1])
    Y_source_org = np.concatenate(Y[ind2take]).reshape(-1)
    X_source_org,Y_source_org,domains_source = balance(X_source_org,Y_source_org, np.concatenate(domains[ind2take]))

    X_target_train_org, Y_target_train_org = X[i][:nb_samples_cal],Y[i][:nb_samples_cal]
    X_target_train_org,Y_target_train_org,domains_target_train = balance(X_target_train_org,Y_target_train_org, domains[i][:nb_samples_cal])

    X_target_test_org, Y_target_test_org = X[i][nb_samples_cal:], Y[i][nb_samples_cal:]
    # X_target_org = xdawncov.transform(X_target_org)
    labels_code_test = labels_codes[i][(n_class*n_cal):]

    print(X_source_org.shape)
    print(labels_code_test.shape)
    print(X_target_train_org.shape)
    print(X_target_test_org.shape)
    X_std = X_source_org.std(axis=0)
    X_source_org /= X_std + 1e-8
    X_std = X_target_train_org.std(axis=0)
    X_target_train_org /= X_std + 1e-8
    X_std = X_target_test_org.std(axis=0)
    X_target_test_org /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    rus = RandomUnderSampler()
    counter=np.array(range(0,len(Y_source_org))).reshape(-1,1)
    index,_ = rus.fit_resample(counter,Y_source_org[:])
    X_source_org = np.squeeze(X_source_org[index,:,:], axis=1)
    Y_source_org = np.squeeze(Y_source_org[index])
    print("balancing the number of ones and zeros")
    rus = RandomUnderSampler()
    counter=np.array(range(0,len(Y_target_train_org))).reshape(-1,1)
    index,_ = rus.fit_resample(counter,Y_target_train_org[:])
    X_target_train_org = np.squeeze(X_target_train_org[index,:,:], axis=1)
    Y_target_train_org = np.squeeze(Y_target_train_org[index])

    print("Creating the dic")
    source_org = {}
    source_org['covs'] = X_source_org
    source_org['labels'] = Y_source_org
    target_org_train = {}
    target_org_train['covs'] = X_target_train_org
    target_org_train['labels'] = Y_target_train_org
    target_org_test = {}
    target_org_test['covs'] = X_target_test_org
    target_org_test['labels'] = Y_target_test_org

    print("Creating the different pipelines")
    lr = 1e-3
    # optimizer = riemannian_adam.RiemannianAdam(learning_rate=lr)
    batchsize = 64 #128 # 64 for burst
    epoch = 20 #45 # 20 for burst
    # clf = SPDNet_AJD(n_epochs=epoch,batch_size=batchsize,valid_split=0.1)
    clf = SPDNet_Tensorflow(bimap_dims=[28,14,7])

    print("Fitting")
    weight_decay = 1e-4
    
    print("get the visualisation")
    R_TSNE_moabb_org = R_TSNE(perplexity = int(0.75*shape_target), verbosity = 1, max_it=200, max_time=2000)
    to_visualize_target = np.concatenate([target_org_test['covs'][:shape_target],target_org_test['covs'][-shape_target:]])
    to_visualize_train = np.concatenate([source_org['covs'][:shape_target],source_org['covs'][-shape_target:]])
    res_tSNE_moabb_target_org = R_TSNE_moabb_org.fit(to_visualize_target)
    res_tSNE_moabb_train_org = R_TSNE_moabb_org.fit(to_visualize_train)
    y_pred['org'].append(TL.get_y_pred_transferlearning(clf, source_org, target_org_train, target_org_test))
    np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/reduced_sets/Testo_S_{}_cov_target_org".format(i),res_tSNE_moabb_target_org)
    np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/reduced_sets/Testo_S_{}_cov_train_org".format(i),res_tSNE_moabb_train_org)

    # get the y_pred with the re-centered matrices
    print("Recenter the martices")
    source_rct, target_rct_train, target_rct_test = TL.RPA_recenter(source_org, target_org_train, target_org_test)
    print("get the visualisation")
    R_TSNE_moabb_rct = R_TSNE(perplexity = int(0.75*shape_target), verbosity = 1, max_it=200, max_time=2000)
    to_visualize_target = np.concatenate([target_rct_test['covs'][:shape_target],target_rct_test['covs'][-shape_target:]])
    to_visualize_train = np.concatenate([source_rct['covs'][:shape_target],source_rct['covs'][-shape_target:]])
    res_tSNE_moabb_target_rct = R_TSNE_moabb_rct.fit(to_visualize_target)
    res_tSNE_moabb_train_rct = R_TSNE_moabb_rct.fit(to_visualize_train)
    y_pred['rct'].append(TL.get_y_pred_transferlearning(clone(clf), source_rct, target_rct_train, target_rct_test))
    np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/reduced_sets/Testo_S_{}_cov_target_rct".format(i),res_tSNE_moabb_target_rct)
    np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/reduced_sets/Testo_S_{}_cov_train_rct".format(i),res_tSNE_moabb_train_rct)

    # rotate the re-centered-stretched matrices using information from classes
    print("Rotating the matices")
    source_rpa, target_rpa_train, target_rpa_test = TL.RPA_rotate(source_rct, target_rct_train, target_rct_test,distance="euc")
    print("get the visualisation")
    R_TSNE_moabb_rpa = R_TSNE(perplexity = int(0.75*shape_target), verbosity = 1, max_it=200, max_time=2000)
    to_visualize_target = np.concatenate([target_rpa_test['covs'][:shape_target],target_rpa_test['covs'][-shape_target:]])
    to_visualize_train = np.concatenate([source_rpa['covs'][:shape_target],source_rpa['covs'][-shape_target:]])
    res_tSNE_moabb_target_rpa = R_TSNE_moabb_rpa.fit(to_visualize_target)
    res_tSNE_moabb_train_rpa = R_TSNE_moabb_rpa.fit(to_visualize_train)
    y_pred['rpa'].append(TL.get_y_pred_transferlearning(clone(clf), source_rpa, target_rpa_train, target_rpa_test))
    np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/reduced_sets/Testo_S_{}_cov_target_rpa".format(i),res_tSNE_moabb_target_rpa)
    np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/reduced_sets/Testo_S_{}_cov_train_rpa".format(i),res_tSNE_moabb_train_rpa)

    for method in methods_list:

        y_pred_temp = y_pred[method]
        # print(y_pred_temp)
        y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred_temp])
        y_test_norm = np.array([0 if y == 0 else 1 for y in Y_target_test_org])

        tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
        spd_accuracy_rpa[method][i] = balanced_accuracy_score(y_test_norm,y_pred_norm)
        

        labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
            y_pred_norm, codes, min_len=30, sfreq=fps, consecutive=50, window_size=window_size
        )
        spd_tps_code_rpa[method][i] = np.mean(mean_long_accumul)
        spd_accuracy_code_rpa[method][i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(spd_accuracy_rpa)
print(spd_tps_code_rpa)
print(spd_accuracy_code_rpa)


TL to the participant :  0
(2400, 32, 32)
(52,)
(320, 32, 32)
(6084, 32, 32)
balancing the number of ones and zeros
balancing the number of ones and zeros
Creating the dic
Creating the different pipelines
Fitting
get the visualisation
Optimizing...


C:\Users\s.velut\Documents\These\riemannian_tSNE\R_TSNE.py:259: UserWarning: Maximum iterations reached.
  warnings.warn("Maximum iterations reached.")


Optimization done in 108.99 seconds.
Optimizing...


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyriemann\utils\distance.py:597: RuntimeWarning: divide by zero encountered in log
  d2 = np.sum(np.log(evals_) ** 2, -1)
C:\Users\s.velut\Documents\These\riemannian_tSNE\R_TSNE.py:149: RuntimeWarning: divide by zero encountered in divide
  return np.sum(P * np.log((P + np.eye(P.shape[0])) / (Q + np.eye(P.shape[0]))))
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyriemann\utils\distance.py:597: RuntimeWarning: invalid value encountered in log
  d2 = np.sum(np.log(evals_) ** 2, -1)
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyriemann\utils\base.py:18: RuntimeWarning: divide by zero encountered in log
  eigvals = operator(eigvals)


ValueError: Matrices must be positive definite. Add regularization to avoid this error.

In [12]:
X_parent.shape

(12, 7020, 32, 32)

: 

#### Test RPA

In [13]:
n_cal = 2
n_class = 4
methods_list = ['org', 'rct', 'rpa', 'clb']
nb_samples_cal = int(n_class*n_cal*(2.2-window_size)*fps)

spd_accuracy_code_rpa = OrderedDict()
spd_accuracy_rpa = OrderedDict()
y_pred = OrderedDict()

spd_tps_code_rpa = OrderedDict()
for method in methods_list:
    spd_accuracy_code_rpa[method] = np.zeros(12)
    spd_accuracy_rpa[method] = np.zeros(12)
    y_pred[method] = []
    spd_tps_code_rpa[method] = np.zeros(12)

shape_target = 300

for i in range(3):
    print("TL to the participant : ", i)
    # ind2take = [2,3]
    X = X_parent.copy()
    Y = Y_parent.copy()
    domains = domains_parent.copy()
    ind2take = [i]
    # ind2take = [j for j in range(3)]

    X_source_org = np.concatenate(X[ind2take]).reshape(-1,X.shape[-2],X.shape[-1])
    Y_source_org = np.concatenate(Y[ind2take]).reshape(-1)
    X_source_org,Y_source_org,domains_source = balance(X_source_org,Y_source_org, np.concatenate(domains[ind2take]))

    X_target_train_org, Y_target_train_org = X[i][:nb_samples_cal],Y[i][:nb_samples_cal]
    X_target_train_org,Y_target_train_org,domains_target_train = balance(X_target_train_org,Y_target_train_org, domains[i][:nb_samples_cal])

    X_target_test_org, Y_target_test_org = X[i][nb_samples_cal:], Y[i][nb_samples_cal:]
    # X_target_org = xdawncov.transform(X_target_org)
    labels_code_test = labels_codes[i][(n_class*n_cal):]

    print(X_source_org.shape)
    print(labels_code_test.shape)
    print(X_target_train_org.shape)
    print(X_target_test_org.shape)
    X_std = X_source_org.std(axis=0)
    X_source_org /= X_std + 1e-8
    X_std = X_target_train_org.std(axis=0)
    X_target_train_org /= X_std + 1e-8
    X_std = X_target_test_org.std(axis=0)
    X_target_test_org /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    rus = RandomUnderSampler()
    counter=np.array(range(0,len(Y_source_org))).reshape(-1,1)
    index,_ = rus.fit_resample(counter,Y_source_org[:])
    X_source_org = np.squeeze(X_source_org[index,:,:], axis=1)
    Y_source_org = np.squeeze(Y_source_org[index])
    print("balancing the number of ones and zeros")
    rus = RandomUnderSampler()
    counter=np.array(range(0,len(Y_target_train_org))).reshape(-1,1)
    index,_ = rus.fit_resample(counter,Y_target_train_org[:])
    X_target_train_org = np.squeeze(X_target_train_org[index,:,:], axis=1)
    Y_target_train_org = np.squeeze(Y_target_train_org[index])

    print("Creating the dic")
    source_org = {}
    source_org['covs'] = X_source_org
    source_org['labels'] = Y_source_org
    target_org_train = {}
    target_org_train['covs'] = X_target_train_org
    target_org_train['labels'] = Y_target_train_org
    target_org_test = {}
    target_org_test['covs'] = X_target_test_org
    target_org_test['labels'] = Y_target_test_org

    print("Creating the different pipelines")
    lr = 1e-3
    # optimizer = riemannian_adam.RiemannianAdam(learning_rate=lr)
    batchsize = 64 #128 # 64 for burst
    epoch = 20 #45 # 20 for burst
    # clf = SPDNet_AJD(n_epochs=epoch,batch_size=batchsize,valid_split=0.1)
    clf = SPDNet_Tensorflow(bimap_dims=[28,14,7])

    print("Fitting")
    weight_decay = 1e-4
    
    print("get the visualisation")
    R_TSNE_moabb_org = R_TSNE(perplexity = int(0.75*shape_target), verbosity = 1, max_it=200, max_time=2000)
    to_visualize_target = np.concatenate([target_org_test['covs'][:shape_target],target_org_test['covs'][-shape_target:]])
    to_visualize_train = np.concatenate([source_org['covs'][:shape_target],source_org['covs'][-shape_target:]])
    res_tSNE_moabb_target_org = R_TSNE_moabb_org.fit(to_visualize_target)
    res_tSNE_moabb_train_org = R_TSNE_moabb_org.fit(to_visualize_train)
    y_pred['org'].append(TL.get_y_pred_transferlearning(clf, source_org, target_org_train, target_org_test))
    np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/reduced_sets/Testo_S_{}_cov_target_org".format(i),res_tSNE_moabb_target_org)
    np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/reduced_sets/Testo_S_{}_cov_train_org".format(i),res_tSNE_moabb_train_org)

    # get the y_pred with the re-centered matrices
    print("Recenter the martices")
    source_rct, target_rct_train, target_rct_test = TL.RPA_recenter(source_org, target_org_train, target_org_test)
    print("get the visualisation")
    R_TSNE_moabb_rct = R_TSNE(perplexity = int(0.75*shape_target), verbosity = 1, max_it=200, max_time=2000)
    to_visualize_target = np.concatenate([target_rct_test['covs'][:shape_target],target_rct_test['covs'][-shape_target:]])
    to_visualize_train = np.concatenate([source_rct['covs'][:shape_target],source_rct['covs'][-shape_target:]])
    res_tSNE_moabb_target_rct = R_TSNE_moabb_rct.fit(to_visualize_target)
    res_tSNE_moabb_train_rct = R_TSNE_moabb_rct.fit(to_visualize_train)
    y_pred['rct'].append(TL.get_y_pred_transferlearning(clone(clf), source_rct, target_rct_train, target_rct_test))
    np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/reduced_sets/Testo_S_{}_cov_target_rct".format(i),res_tSNE_moabb_target_rct)
    np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/reduced_sets/Testo_S_{}_cov_train_rct".format(i),res_tSNE_moabb_train_rct)

    # rotate the re-centered-stretched matrices using information from classes
    print("Rotating the matices")
    source_rpa, target_rpa_train, target_rpa_test = TL.RPA_rotate(source_rct, target_rct_train, target_rct_test,distance="euc")
    print("get the visualisation")
    R_TSNE_moabb_rpa = R_TSNE(perplexity = int(0.75*shape_target), verbosity = 1, max_it=200, max_time=2000)
    to_visualize_target = np.concatenate([target_rpa_test['covs'][:shape_target],target_rpa_test['covs'][-shape_target:]])
    to_visualize_train = np.concatenate([source_rpa['covs'][:shape_target],source_rpa['covs'][-shape_target:]])
    res_tSNE_moabb_target_rpa = R_TSNE_moabb_rpa.fit(to_visualize_target)
    res_tSNE_moabb_train_rpa = R_TSNE_moabb_rpa.fit(to_visualize_train)
    y_pred['rpa'].append(TL.get_y_pred_transferlearning(clone(clf), source_rpa, target_rpa_train, target_rpa_test))
    np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/reduced_sets/Testo_S_{}_cov_target_rpa".format(i),res_tSNE_moabb_target_rpa)
    np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/reduced_sets/Testo_S_{}_cov_train_rpa".format(i),res_tSNE_moabb_train_rpa)

    for method in methods_list:

        y_pred_temp = y_pred[method]
        # print(y_pred_temp)
        y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred_temp])
        y_test_norm = np.array([0 if y == 0 else 1 for y in Y_target_test_org])

        tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
        spd_accuracy_rpa[method][i] = balanced_accuracy_score(y_test_norm,y_pred_norm)
        

        labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
            y_pred_norm, codes, min_len=30, sfreq=fps, consecutive=50, window_size=window_size
        )
        spd_tps_code_rpa[method][i] = np.mean(mean_long_accumul)
        spd_accuracy_code_rpa[method][i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(spd_accuracy_rpa)
print(spd_tps_code_rpa)
print(spd_accuracy_code_rpa)


TL to the participant :  0
(2400, 32, 32)
(52,)
(320, 32, 32)
(6084, 32, 32)
balancing the number of ones and zeros
balancing the number of ones and zeros
Creating the dic
Creating the different pipelines
Fitting
get the visualisation
Optimizing...


C:\Users\s.velut\Documents\These\riemannian_tSNE\R_TSNE.py:259: UserWarning: Maximum iterations reached.
  warnings.warn("Maximum iterations reached.")


Optimization done in 108.99 seconds.
Optimizing...


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyriemann\utils\distance.py:597: RuntimeWarning: divide by zero encountered in log
  d2 = np.sum(np.log(evals_) ** 2, -1)
C:\Users\s.velut\Documents\These\riemannian_tSNE\R_TSNE.py:149: RuntimeWarning: divide by zero encountered in divide
  return np.sum(P * np.log((P + np.eye(P.shape[0])) / (Q + np.eye(P.shape[0]))))
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyriemann\utils\distance.py:597: RuntimeWarning: invalid value encountered in log
  d2 = np.sum(np.log(evals_) ** 2, -1)
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyriemann\utils\base.py:18: RuntimeWarning: divide by zero encountered in log
  eigvals = operator(eigvals)


ValueError: Matrices must be positive definite. Add regularization to avoid this error.

### RPA from utils functions

In [4]:
subjects = [1,2,3,4,5,6,7,8,9,10,11,12]

In [5]:
raw_data,labels,codes,labels_codes = get_BVEP_data(subjects,True)

Choosing the first None classes from all possible events.


None
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped


C:\Users\s.velut\Documents\These\moabb\moabb\datasets\castillos2023.py:182: RuntimeWarning: Data file name in EEG.data (P13_burst100.fdt) is incorrect, the file name must have changed on disk, using the correct file name (P6_burst100.fdt).
  raw = mne.io.read_raw_eeglab(file_path_list[0],preload=True, verbose=False)


Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 60 events and 1101 original time points ...
0 bad epochs dropped
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower pas

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


...

0 bad epochs dropped
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 1651 samples (3.302 s)

7020 events found
Event IDs: [100 101]
Not setting metadata
7020 matching events found
Setting baseline interval to [-0.01, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 7020 events and 131 original time points ...


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


0 bad epochs dropped
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 1651 samples (3.302 s)

7020 events found
Event IDs: [100 101]
Not setting metadata
7020 matching events found
Setting baseline interval to [-0.01, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 7020 events and 131 original time points ...


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


0 bad epochs dropped


In [14]:
X_parent,Y_parent,domains_parent = prepare_data(subjects,raw_data,labels,True,True)

In [9]:
X_parent.shape

(12, 7020, 32, 32)

In [ ]:
n_cal = 1
n_class = 4
window_size = 0.25
nb_frames_windows = int(n_cal*n_class*(2.2-window_size)*fps)
# spd_accuracy_code_perso = np.zeros(12)
# spd_tps_code_perso = np.zeros(12)
# spd_accuracy_perso = np.zeros(12)
methods_list = ['org', 'rct', 'rpa', 'clb']
spd_accuracy_code_perso = OrderedDict()
spd_accuracy_perso = OrderedDict()
spd_tps_code_perso = OrderedDict()
for method in methods_list:
    spd_accuracy_code_perso[method] = np.zeros(12)
    spd_accuracy_perso[method] = np.zeros(12)
    spd_tps_code_perso[method] = np.zeros(12)

for i in range(0,12):
    print("TL to the participant : ", i)
    X = X_parent.copy()
    Y = Y_parent.copy()
    # ind2take = [1,2,3]
    ind2take = [j for j in range(12) if j!=i]

    X_source_org = X[ind2take].reshape(-1,X[i].shape[-2],X[i].shape[-1])
    Y_source_org = Y[ind2take].reshape(-1)

    X_target_train_org = X[i][:nb_frames_windows]
    Y_target_train_org = Y[i][:nb_frames_windows]

    X_target_test_org = X[i][nb_frames_windows:]
    Y_target_test_org = Y[i][nb_frames_windows:]
    labels_code_test = labels_codes[i][n_cal*n_class:]

    X_std = X_source_org.std(axis=0)
    X_source_org /= X_std + 1e-8
    X_std = X_target_train_org.std(axis=0)
    X_target_train_org /= X_std + 1e-8
    X_std = X_target_test_org.std(axis=0)
    X_target_test_org /= X_std + 1e-8
    
    X_source_org, Y_source_org,_ = balance(X_source_org,Y_source_org)
    X_target_train_org, Y_target_train_org,_ = balance(X_target_train_org,Y_target_train_org)
    
    print(X_source_org.shape)
    print(X_target_train_org.shape)
    print(X_target_test_org.shape)
    print("Creating the dic")
    source_org = {}
    source_org['covs'] = X_source_org
    source_org['labels'] = Y_source_org
    target_org_train = {}
    target_org_train['covs'] = X_target_train_org
    target_org_train['labels'] = Y_target_train_org
    target_org_test = {}
    target_org_test['covs'] = X_target_test_org
    target_org_test['labels'] = Y_target_test_org
    

    print("Creating the different pipelines")
    lr = 1e-3
    # optimizer = riemannian_adam.RiemannianAdam(learning_rate=lr)
    batchsize = 64 #128 # 64 for burst
    epoch = 20 #45 # 20 for burst
    # clf = SPDNet_AJD(n_epochs=epoch,batch_size=batchsize,valid_split=0.1)
    clf = SPDNet_Tensorflow(bimap_dims=[28,14,7])

    print("Fitting")
    weight_decay = 1e-4

    # X_source_org, x_val, Y_source_org, y_val = train_test_split(X_source_org, Y_source_org, test_size=0.1, random_state=42, shuffle=True)
    # history = clf.fit(np.array(X_source_org), Y_source_org,
    #                 batch_size=batchsize, epochs=epoch,
    #                 validation_data=(np.array(x_val), y_val), shuffle=True)

    # # clf.fit_and_predict(X_source_org,Y_source_org,X_target_org,Y_test,"C:\\Users\\s.velut\\Documents\\These\\Protheus_PHD\\pictures\\SPD_Net")

    # print("getting accuracy of participant ", i)
    # y_pred = clf.predict(X_target_org)[:]
    # y_pred = np.array(y_pred)
    y_pred = get_ypred(source_org,target_org_train,target_org_test,clf)
    

    for method in methods_list:
        y_pred_temp = y_pred[method]
        # print(y_pred_temp)
        y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred_temp[0]])
        y_test_norm = np.array([0 if y == 0 else 1 for y in Y_target_test_org])

        spd_accuracy_perso[method][i] = balanced_accuracy_score(y_test_norm,y_pred_norm)
        print(spd_accuracy_perso[method][i])
        # tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
        # spd_accuracy_perso[method][i] = (tp+tn)/len(y_test_norm)

        labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
            y_pred_norm, codes, min_len=30, sfreq=fps, consecutive=50, window_size=window_size
        )
        spd_tps_code_perso[method][i] = np.mean(mean_long_accumul)
        spd_accuracy_code_perso[method][i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(spd_accuracy_perso)
print(spd_tps_code_perso)
print(spd_accuracy_code_perso)

In [ ]:
Y_source_org

In [ ]:
for x in X[1][:10]:
    print(np.all(np.linalg.eigvals(x) > 0))

### RECENTER from Bruna

In [27]:
from Scripts.Alignments import transform,riemannian

In [39]:
n_cal = 1
n_class = 4
window_size = 0.25
nb_frames_windows = int(n_cal*n_class*(2.2-window_size)*fps)
# spd_accuracy_code_perso = np.zeros(12)
# spd_tps_code_perso = np.zeros(12)
# spd_accuracy_perso = np.zeros(12)
methods_list = ['org', 'rct']
spd_accuracy_code_perso = OrderedDict()
spd_accuracy_perso = OrderedDict()
spd_tps_code_perso = OrderedDict()
for method in methods_list:
    spd_accuracy_code_perso[method] = np.zeros(12)
    spd_accuracy_perso[method] = np.zeros(12)
    spd_tps_code_perso[method] = np.zeros(12)

for i in range(1,12):
    print("TL to the participant : ", i)
    y_pred = []

    X = X_parent.copy()
    Y = Y_parent.copy()
    domains = domains_parent.copy()
    # ind2take = [1,2,3]
    ind2take = [j for j in range(12) if j!=i]

    X_source_org = X[ind2take].reshape(-1,X[i].shape[-2],X[i].shape[-1])
    Y_source_org = Y[ind2take].reshape(-1)
    domains_source = domains[ind2take].reshape(-1)

    X_target_train_org = X[i][:nb_frames_windows]
    Y_target_train_org = Y[i][:nb_frames_windows]
    domains_target_train = domains[i][:nb_frames_windows].reshape(-1)

    X_target_test_org = X[i][nb_frames_windows:]
    Y_target_test_org = Y[i][nb_frames_windows:]
    domains_target_test = domains[i][nb_frames_windows:].reshape(-1)
    labels_code_test = labels_codes[i][n_cal*n_class:]

    X_std = X_source_org.std(axis=0)
    X_source_org /= X_std + 1e-8
    X_std = X_target_train_org.std(axis=0)
    X_target_train_org /= X_std + 1e-8
    X_std = X_target_test_org.std(axis=0)
    X_target_test_org /= X_std + 1e-8
    
    X_source_org, Y_source_org,domains_source = balance(X_source_org,Y_source_org,domains_source)
    X_target_train_org, Y_target_train_org,domains_target_train = balance(X_target_train_org,Y_target_train_org,domains_target_train)
    
    print(X_source_org.shape)
    print(X_target_train_org.shape)
    print(X_target_test_org.shape)
    print("Creating the dic")

    print("Creating the different pipelines org")
    lr = 1e-3
    # optimizer = riemannian_adam.RiemannianAdam(learning_rate=lr)
    batchsize = 64 #128 # 64 for burst
    epoch = 20 #45 # 20 for burst
    # clf = SPDNet_AJD(n_epochs=epoch,batch_size=batchsize,valid_split=0.1)
    clf = SPDNet_Tensorflow(bimap_dims=[28,14,7])

    Y_source_org = np.concatenate([Y_source_org,Y_target_train_org])
    X_source_org = np.concatenate([X_source_org,X_target_train_org])

    X_source_train, x_val, Y_source_train, y_val = train_test_split(X_source_org, Y_source_org, test_size=0.1, random_state=42, shuffle=True)
    history = clf.fit(np.array(X_source_train), Y_source_train,
                    batch_size=batchsize, epochs=epoch,
                    validation_data=(np.array(x_val), y_val), shuffle=True)

    print("getting accuracy of participant org", i)
    y_pred_org = clf.predict(X_target_test_org)[:]
    y_pred.append(np.array(y_pred_org))

    print("Recentering")
    tfRR = transform.TransformRR()
    print(np.concatenate([X_source_org,X_target_test_org]).shape)
    print(np.concatenate([domains_source,domains_target_train,domains_target_test]).shape)
    Aligned_Cov = tfRR.transform([np.concatenate([X_source_org,X_target_test_org]),np.concatenate([domains_source,domains_target_train,domains_target_test])])
    
    n = X_source_org.shape[0]
    X_source_rct = Aligned_Cov[:n]
    Y_source_rct = Y_source_org
    X_target_rct = Aligned_Cov[n:]

    X_source_train, x_val, Y_source_train, y_val = train_test_split(X_source_rct, Y_source_rct, test_size=0.1, random_state=42, shuffle=True)
    history = clf.fit(np.array(X_source_train), Y_source_train,
                    batch_size=batchsize, epochs=epoch,
                    validation_data=(np.array(x_val), y_val), shuffle=True)

    print("getting accuracy of participant rct", i)
    y_pred_rct = clf.predict(X_target_rct)[:]
    y_pred.append(np.array(y_pred_rct))    
    

    for k,method in enumerate(methods_list):
        y_pred_temp = y_pred[k]
        # print(y_pred_temp)
        y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred_temp])
        y_test_norm = np.array([0 if y == 0 else 1 for y in Y_target_test_org])

        spd_accuracy_perso[method][i] = balanced_accuracy_score(y_test_norm,y_pred_norm)
        print(spd_accuracy_perso[method][i])
        # tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
        # spd_accuracy_perso[method][i] = (tp+tn)/len(y_test_norm)

        labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
            y_pred_norm, codes, min_len=30, sfreq=fps, consecutive=50, window_size=window_size
        )
        spd_tps_code_perso[method][i] = np.mean(mean_long_accumul)
        spd_accuracy_code_perso[method][i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(spd_accuracy_perso)
print(spd_tps_code_perso)
print(spd_accuracy_code_perso)

TL to the participant :  1
(26400, 32, 32)
(160, 32, 32)
(6552, 32, 32)
Creating the dic
Creating the different pipelines org
Epoch 1/20
374/374 [==============================] - 6s 15ms/step - loss: 0.5227 - sparse_categorical_accuracy: 0.7610 - val_loss: 0.4125 - val_sparse_categorical_accuracy: 0.8238
Epoch 2/20
374/374 [==============================] - 5s 14ms/step - loss: 0.3988 - sparse_categorical_accuracy: 0.8272 - val_loss: 0.3925 - val_sparse_categorical_accuracy: 0.8298
Epoch 3/20
374/374 [==============================] - 5s 14ms/step - loss: 0.3884 - sparse_categorical_accuracy: 0.8298 - val_loss: 0.3806 - val_sparse_categorical_accuracy: 0.8351
Epoch 4/20
374/374 [==============================] - 5s 14ms/step - loss: 0.3851 - sparse_categorical_accuracy: 0.8303 - val_loss: 0.3799 - val_sparse_categorical_accuracy: 0.8332
Epoch 5/20
374/374 [==============================] - 5s 14ms/step - loss: 0.3833 - sparse_categorical_accuracy: 0.8327 - val_loss: 0.3818 - val_spars

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  2
(26400, 32, 32)
(160, 32, 32)
(6552, 32, 32)
Creating the dic
Creating the different pipelines org
Epoch 1/20
374/374 [==============================] - 7s 17ms/step - loss: 0.5836 - sparse_categorical_accuracy: 0.7319 - val_loss: 0.4314 - val_sparse_categorical_accuracy: 0.8133
Epoch 2/20
374/374 [==============================] - 6s 17ms/step - loss: 0.4109 - sparse_categorical_accuracy: 0.8185 - val_loss: 0.4173 - val_sparse_categorical_accuracy: 0.8057
Epoch 3/20
374/374 [==============================] - 6s 16ms/step - loss: 0.4010 - sparse_categorical_accuracy: 0.8222 - val_loss: 0.4043 - val_sparse_categorical_accuracy: 0.8121
Epoch 4/20
374/374 [==============================] - 6s 16ms/step - loss: 0.3978 - sparse_categorical_accuracy: 0.8240 - val_loss: 0.4032 - val_sparse_categorical_accuracy: 0.8121
Epoch 5/20
374/374 [==============================] - 6s 16ms/step - loss: 0.3952 - sparse_categorical_accuracy: 0.8251 - val_loss: 0.4270 - val_spars

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  3
(26400, 32, 32)
(160, 32, 32)
(6552, 32, 32)
Creating the dic
Creating the different pipelines org
Epoch 1/20
374/374 [==============================] - 6s 15ms/step - loss: 0.5471 - sparse_categorical_accuracy: 0.7382 - val_loss: 0.4368 - val_sparse_categorical_accuracy: 0.8005
Epoch 2/20
374/374 [==============================] - 6s 16ms/step - loss: 0.4034 - sparse_categorical_accuracy: 0.8227 - val_loss: 0.3890 - val_sparse_categorical_accuracy: 0.8328
Epoch 3/20
374/374 [==============================] - 6s 16ms/step - loss: 0.3926 - sparse_categorical_accuracy: 0.8245 - val_loss: 0.3764 - val_sparse_categorical_accuracy: 0.8362
Epoch 4/20
374/374 [==============================] - 6s 16ms/step - loss: 0.3891 - sparse_categorical_accuracy: 0.8269 - val_loss: 0.3969 - val_sparse_categorical_accuracy: 0.8227
Epoch 5/20
374/374 [==============================] - 6s 16ms/step - loss: 0.3864 - sparse_categorical_accuracy: 0.8261 - val_loss: 0.3914 - val_spars

...

TL to the participant :  10
(26400, 32, 32)
(160, 32, 32)
(6552, 32, 32)
Creating the dic
Creating the different pipelines org
Epoch 1/20
374/374 [==============================] - 7s 16ms/step - loss: 0.5510 - sparse_categorical_accuracy: 0.7428 - val_loss: 0.4312 - val_sparse_categorical_accuracy: 0.8197
Epoch 2/20
374/374 [==============================] - 6s 16ms/step - loss: 0.4005 - sparse_categorical_accuracy: 0.8282 - val_loss: 0.3968 - val_sparse_categorical_accuracy: 0.8257
Epoch 3/20
374/374 [==============================] - 6s 16ms/step - loss: 0.3863 - sparse_categorical_accuracy: 0.8310 - val_loss: 0.3915 - val_sparse_categorical_accuracy: 0.8204
Epoch 4/20
374/374 [==============================] - 6s 16ms/step - loss: 0.3822 - sparse_categorical_accuracy: 0.8322 - val_loss: 0.3892 - val_sparse_categorical_accuracy: 0.8234
Epoch 5/20
374/374 [==============================] - 6s 16ms/step - loss: 0.3801 - sparse_categorical_accuracy: 0.8345 - val_loss: 0.3981 - val_spar

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  11
(26400, 32, 32)
(160, 32, 32)
(6552, 32, 32)
Creating the dic
Creating the different pipelines org
Epoch 1/20
374/374 [==============================] - 7s 16ms/step - loss: 0.5787 - sparse_categorical_accuracy: 0.7338 - val_loss: 0.4292 - val_sparse_categorical_accuracy: 0.8242
Epoch 2/20
374/374 [==============================] - 6s 15ms/step - loss: 0.3992 - sparse_categorical_accuracy: 0.8290 - val_loss: 0.3927 - val_sparse_categorical_accuracy: 0.8257
Epoch 3/20
374/374 [==============================] - 6s 15ms/step - loss: 0.3828 - sparse_categorical_accuracy: 0.8302 - val_loss: 0.3816 - val_sparse_categorical_accuracy: 0.8340
Epoch 4/20
374/374 [==============================] - 6s 15ms/step - loss: 0.3798 - sparse_categorical_accuracy: 0.8315 - val_loss: 0.3798 - val_sparse_categorical_accuracy: 0.8377
Epoch 5/20
374/374 [==============================] - 6s 15ms/step - loss: 0.3770 - sparse_categorical_accuracy: 0.8322 - val_loss: 0.3719 - val_spar

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


OrderedDict([('org', array([0.        , 0.77919275, 0.84620306, 0.80266937, 0.83196337,
       0.87780744, 0.79771263, 0.868368  , 0.73197717, 0.92614138,
       0.75768594, 0.73275957])), ('rct', array([0.        , 0.50363586, 0.50679768, 0.49830173, 0.50789304,
       0.4980532 , 0.49963181, 0.50067194, 0.48703516, 0.51356314,
       0.49982511, 0.49721097]))])
OrderedDict([('org', array([0.        , 1.44878049, 1.44795918, 1.50189394, 1.4218254 ,
       1.41870748, 1.4468254 , 1.42421384, 1.42702703, 1.45416667,
       1.46111111, 1.48619048])), ('rct', array([0.        , 1.43      , 1.37878788, 1.37892157, 1.34087302,
       1.42837838, 1.3452381 , 1.33333333, 1.43425926, 1.484375  ,
       1.41153846, 1.38958333]))])
OrderedDict([('org', array([0.  , 0.91, 0.98, 0.89, 0.95, 0.96, 0.86, 0.98, 0.84, 1.  , 0.7 ,
       0.79])), ('rct', array([0.  , 0.27, 0.25, 0.29, 0.21, 0.27, 0.21, 0.25, 0.27, 0.16, 0.18,
       0.27]))])


### Get TSNE representation

In [11]:
def visualize(domains_source,domain_target,shape2take,cov_source,covmat_target,sub,prefix):
    for d in np.unique(domains_source)[:4]:
        covmat_source = cov_source.copy()
        print("get the visualisation for ",d)
        R_TSNE_moabb = R_TSNE(perplexity = int(1.5*shape2take), verbosity = 1, max_it=200, max_time=2000)
        if shape2take<=0:
            to_visualize_train = covmat_source['covs'][domains_source==d]
        else:
            to_visualize_train = np.concatenate([covmat_source['covs'][domains_source==d][:shape2take],covmat_source['covs'][domains_source==d][-shape2take:]])
        res_tSNE_moabb_train_org = R_TSNE_moabb.fit(to_visualize_train)
        np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/reduced_sets/S{}_D{}_cov_train{}".format(sub,d,prefix),res_tSNE_moabb_train_org)
    if shape2take<=0:
        to_visualize_target = covmat_target['covs']
    else:
        to_visualize_target = np.concatenate([covmat_target['covs'][:shape2take],covmat_target['covs'][-shape2take:]])
    res_tSNE_moabb_target_org = R_TSNE_moabb.fit(to_visualize_target)
    np.save("C:/Users/s.velut/Documents/These/Protheus_PHD/results/reduced_sets/S{}_D{}_cov_target{}".format(sub,domain_target,prefix),res_tSNE_moabb_target_org)


In [12]:
def get_TSNE_representation(source_org,target_org_train,target_org_test,sub,domains_source,domain_target):
    # create a scores dictionary
    methods_list = ['org', 'rct', 'rpa', 'clb']
    y_pred = OrderedDict()
    for method in methods_list:
        y_pred[method] = []

    shape2take = 400

    # get the split for the source and target dataset
    # source_org, target_org_train, target_org_test = TL.get_sourcetarget_split(source, target, ncovs_train=10)

    # get the y_pred with the original dataset
    print("get the visualisation")
    visualize(domains_source,domain_target,shape2take,source_org,target_org_test,sub,"_org")

    # get the y_pred with the re-centered matrices
    print("Recenter the martices")
    source_rct, target_rct_train, target_rct_test = TL.RPA_recenter(source_org, target_org_train, target_org_test)
    print("get the visualisation")
    visualize(domains_source,domain_target,shape2take,source_rct,target_rct_test,sub,"_rct")

    # rotate the re-centered-stretched matrices using information from classes
    print("Rotating the matices")
    source_rpa, target_rpa_train, target_rpa_test = TL.RPA_rotate(source_rct, target_rct_train, target_rct_test,distance="rie")
    print("get the visualisation")
    visualize(domains_source,domain_target,shape2take,source_rpa,target_rpa_test,sub,"_rpa")
        

In [13]:
n_cal = 1
n_class = 4
window_size = 0.25
nb_frames_windows = int(n_cal*n_class*(2.2-window_size)*fps)
# spd_accuracy_code_perso = np.zeros(12)
# spd_tps_code_perso = np.zeros(12)
# spd_accuracy_perso = np.zeros(12)
methods_list = ['org', 'rct', 'rpa', 'clb']

for i in range(2):
    print("TL to the participant : ", i)
    # ind2take = [1,2,3]
    ind2take = [j for j in range(12) if j!=i]
    X = X_parent.copy()
    Y = Y_parent.copy()
    domains = domains_parent.copy()

    X_source_org = X[ind2take].reshape(-1,X[i].shape[-2],X[i].shape[-1])
    Y_source_org = labels[ind2take].reshape(-1)
    domains_source = domains[ind2take].reshape(-1)

    X_target_train_org = X[i][:nb_frames_windows]
    Y_target_train_org = labels[ i][:nb_frames_windows]
    domains_target = domains[ i][:nb_frames_windows]

    X_target_test_org = X[i][nb_frames_windows:]
    Y_target_test_org = labels[i][nb_frames_windows:]
    labels_code_test = labels_codes[i][n_cal*n_class:]

    X_std = X_source_org.std(axis=0)
    X_source_org /= X_std + 1e-8
    X_std = X_target_train_org.std(axis=0)
    X_target_train_org /= X_std + 1e-8
    X_std = X_target_test_org.std(axis=0)
    X_target_test_org /= X_std + 1e-8
    
    X_source_org, Y_source_org,domains_source = balance(X_source_org,Y_source_org,domains_source)
    X_target_train_org, Y_target_train_org,domains_target = balance(X_target_train_org,Y_target_train_org,domains_target)
    
    print(X_source_org.shape)
    print(X_target_train_org.shape)
    print(X_target_test_org.shape)
    print("Creating the dic")
    source_org = {}
    source_org['covs'] = X_source_org
    source_org['labels'] = Y_source_org
    target_org_train = {}
    target_org_train['covs'] = X_target_train_org
    target_org_train['labels'] = Y_target_train_org
    target_org_test = {}
    target_org_test['covs'] = X_target_test_org
    target_org_test['labels'] = Y_target_test_org
    
    get_TSNE_representation(source_org,target_org_train,target_org_test,i,domains_source,domains_target[0])


TL to the participant :  0
(26400, 32, 32)
(160, 32, 32)
(6552, 32, 32)
Creating the dic
get the visualisation
get the visualisation for  Source_sub_1
shape X in distance (800, 32, 32)
there is inf in X: False
there is nan in X: False
Optimizing...
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shape X in di

C:\Users\s.velut\Documents\These\riemannian_tSNE\R_TSNE.py:259: UserWarning: Maximum iterations reached.
  warnings.warn("Maximum iterations reached.")


Optimization done in 169.01 seconds.
get the visualisation for  Source_sub_10
shape X in distance (800, 32, 32)
there is inf in X: False
there is nan in X: False
Optimizing...
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shap

C:\Users\s.velut\Documents\These\riemannian_tSNE\R_TSNE.py:259: UserWarning: Maximum iterations reached.
  warnings.warn("Maximum iterations reached.")


Optimization done in 175.34 seconds.
get the visualisation for  Source_sub_10
shape X in distance (800, 32, 32)
there is inf in X: False
there is nan in X: False
Optimizing...
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shape X in distance (800, 2, 2)
there is inf in X: False
there is nan in X: False
shap

In [35]:
print("shape X in distance",X.shape)
print("there is inf in X:",np.isinf(X).any())
print("there is nan in X:",np.isnan(X).any())

(26400, 32, 32)


### Plot

In [ ]:
# plt.plot(accuracy_code_looa,label='looa')
# plt.plot(accuracy_code_adap,label='adap')
plt.plot(spd_accuracy_code_perso,label='perso')
plt.legend()
plt.show()

### SPDNET on frame

In [ ]:
n_cal = 7
n_class = 4
spd_accuracy_code_perso = np.zeros(12)
spd_tps_code_perso = np.zeros(12)
spd_accuracy_perso = np.zeros(12)

for i in range(12):
    print("TL to the participant : ", i)
    X_train = raw_data[i][:int((2.2-window_size)*n_class*n_cal*60)]
    Y_train = labels[i][:int((2.2-window_size)*n_class*n_cal*60)]
    xdawncov = XdawnCovariances(estimator="lwf",xdawn_estimator="lwf",nfilter=8)
    X_train = xdawncov.fit_transform(X_train,Y_train)
    
    Y_test = labels[i][int((2.2-window_size)*n_class*n_cal*60):]
    X_test = raw_data[i][int((2.2-window_size)*n_class*n_cal*60):]
    X_test = xdawncov.transform(X_test)
    labels_code_test = labels_code[i][n_cal*4:]

    print(X_train.shape)
    print(X_test.shape)
    X_std = X_train.std(axis=0)
    X_train /= X_std + 1e-8
    X_std = X_test.std(axis=0)
    X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    rus = RandomUnderSampler()
    counter=np.array(range(0,len(Y_train))).reshape(-1,1)
    index,_ = rus.fit_resample(counter,Y_train[:])
    X_train = np.squeeze(X_train[index,:,:], axis=1)
    Y_train = np.squeeze(Y_train[index])

    print("Creating the different pipelines")
    lr = 1e-3
    # optimizer = riemannian_adam.RiemannianAdam(learning_rate=lr)
    batchsize = 64 #128 # 64 for burst
    epoch = 20 #45 # 20 for burst
    # clf = SPDNet_AJD(n_epochs=epoch,batch_size=batchsize,valid_split=0.1)
    clf = SPDNet_Tensorflow(bimap_dims=[30,15,8])

    print("Fitting")
    weight_decay = 1e-4
    
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42, shuffle=True)
    history = clf.fit(np.array(x_train), y_train,
                    batch_size=batchsize, epochs=epoch,
                    validation_data=(np.array(x_val), y_val), shuffle=True)

    # clf.fit_and_predict(X_train,Y_train,X_test,Y_test,"C:\\Users\\s.velut\\Documents\\These\\Protheus_PHD\\pictures\\SPD_Net")
    
    print("getting accuracy of participant ", i)
    y_pred = clf.predict(X_test)[:]
    y_pred = np.array(y_pred)
    y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
    y_test_norm = np.array([0 if y == 0 else 1 for y in Y_test])

    tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
    spd_accuracy_perso[i] = (tp+tn)/len(y_test_norm)

    labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
        y_pred_norm, codes, min_len=30, sfreq=60, consecutive=50, window_size=window_size
    )
    spd_tps_code_perso[i] = np.mean(mean_long_accumul)
    spd_accuracy_code_perso[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(spd_accuracy_perso)
print(spd_tps_code_perso)
print(spd_accuracy_code_perso)

## Train test ERP TS+LDA

In [ ]:
def create_template(trial_len,code,sfreq,fps):
    y = np.zeros(trial_len)
    code_pos = 0
    for idx in range(trial_len):
        if idx/sfreq >= (code_pos+1)/fps:
            code_pos += 1 
        y[idx] = int(code[code_pos])

    return y

In [ ]:
# Get target frequency list
event_id = epochs.event_id
peaks = [float(peak) for peak in codes.keys()]  # HERE

# Get lenght of the wave (length of trials)
trial_len = raw_data_dl.shape[-1]  # HERE

# Make time points 't'
t = np.arange(0, trial_len / sfreq, 1 / sfreq)  # HERE

# Create empty array
n_class = len(peaks)  # HERE
ref_signals = np.zeros((n_class, trial_len))  # HERE

# Iterate over peaks (the index will be needed to add waves to ref_signals at the end
for class_idx, peak in enumerate(peaks):
    y = create_template(trial_len,codes[class_idx],sfreq,60)
    
    # Add to the empty array
    ref_signals[class_idx, :] = y  # HERE

In [ ]:
ref_signals[labels_dl[0][0:28], :].T.shape

In [ ]:
# Empty list for predictions
y_pred = []

# CCA model
cca = CCA(n_components=1, max_iter=1000)

# Loop over trials
n_trials, _, _ = raw_data_dl[0].shape


for trial_n in range(n_trials):
    # Empty list to store the correlations
    corrs = []
    next_trial = raw_data_dl[0][trial_n, ...]

    # Iterate over classes
    for class_idx in range(len(peaks)):
        # Get the corresponding template
        template = ref_signals[class_idx, :]

        # Fit CCA and transform
        cca.fit(next_trial.T, template.T)
        x_scores, y_scores = cca.fit_transform(next_trial.T, template.T)

        # Get correlation
        corr_score = np.corrcoef(x_scores, y_scores, rowvar=False)[0, 1]
        corrs.append(corr_score)
    
    # Append the label of the max correlation to the pred list
    trial_pred = np.argmax(corrs)
    y_pred.append(trial_pred)

# Get accuracy
acc_score = accuracy_score(labels_dl[0], y_pred)
print(f'Total accuracy score: {acc_score}')

In [ ]:
n_cal = 7
accuracy_code_perso = np.zeros(12)
tps_code_perso = np.zeros(12)
accuracy_perso = np.zeros(12)

for i in range(12):
    print("TL to the participant : ", i)
    X_train = raw_data_dl[i][:n_class*n_cal]
    Y_train = labels_dl[i][:n_class*n_cal]
    
    Y_test = labels_dl[i][n_class*n_cal:]
    X_test = raw_data_dl[i][n_class*n_cal:]
    labels_code_test = Y_test

    print(X_train.shape)
    print(X_test.shape)
    X_std = X_train.std(axis=0)
    X_train /= X_std + 1e-8
    X_std = X_test.std(axis=0)
    X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    rus = RandomUnderSampler()
    counter=np.array(range(0,len(Y_train))).reshape(-1,1)
    index,_ = rus.fit_resample(counter,Y_train[:])
    X_train = np.squeeze(X_train[index,:,:], axis=1)
    Y_train = np.squeeze(Y_train[index])

    print("Creating the different pipelines")
    clf = make_pipeline(XdawnCovariances(nfilter=8, estimator="oas", xdawn_estimator="lwf"),
            TangentSpace(),
            LDA(solver="lsqr", shrinkage="auto"))
    
    print("Fitting")
    clf = clf.fit(np.array(X_train), Y_train)
    y_pred = clf.predict(X_test)
    print("getting accuracy of participant ", i)
    accuracy_perso[i] = balanced_accuracy_score(Y_test,y_pred)

    labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
            y_pred, dataset_moabb.codes, min_len=30, sfreq=60, consecutive=50, window_size=window_size
    )
    tps_code_perso[i] = np.mean(mean_long_accumul)
    accuracy_code_perso[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(accuracy_perso)
print(tps_code_perso)
print(accuracy_code_perso)